# Validating Scaling Arguments Numerically

Theoretical scaling calculations suggest that self-similar scaling is possible within various absorber models. In this notebook, we will numerically solve these models to confirm that self-similar scaling does in fact exist.

## Physical (non-reactive) solvent.

We begin by testing a non-reactive (i.e. a physical) solvent. This is described by the following ODE's:

$$
v_G \frac{dc_G}{dz} = -kac_G
$$

$$
v_L \frac{dc_L}{dz} = -kac_G
$$

$$
v_Gc_P^G\rho_G \frac{dT^G}{dz} = -ha(T^G-T^L) 
$$

$$
v_Lc_P^L\rho_L \frac{dT^L}{dz} = -ha(T^G-T^L) + kac_G\Delta H
$$

The boundary conditions are:

$$
    c_G|_{z=0} = c_G^{\infty}
$$

$$
    c_L|_{z=L} = c_L^{\infty}
$$

$$
    T_G|_{z=0} = T^{G,\infty}
$$

$$
    T_L|_{z=L} = T^{L,\infty}
$$

Both $h$ and $a$ are given by correlations from the Onda paper:

$$
    \frac{a}{a_t} = 1 - \exp \left(-1.45 
    \left(\frac{\sigma_c}{\sigma}\right)^{0.75} \mathrm{Re_L}^{0.1} 
    \mathrm{Fr_L}^{-0.05} \mathrm{We_L}^{0.2} \right)
$$

Expanding this out...


$$
    \frac{a}{a_t} = 1 - \exp \left(-1.45 
    \left(\frac{\sigma_c}{\sigma}\right)^{0.75} \left(\frac{\rho_Lv_L}{a_t\mu_L} 
    \right)^{0.1} \left(\frac{v_L^2 a_t}{g} \right)^{-0.05} 
    \left(\frac{\rho_Lv_L^2}{\sigma a_t}\right)^{0.2} \right)
$$

and
$$
\mathrm{Nu}_G = 5.23 \mathrm{Re}_G^{0.7} \mathrm{Pr}_G^{1/3}
$$

where
$$
\mathrm{Nu}_G = \frac{h}{\kappa_Ga_t}; \quad \mathrm{Re}_G = 
\frac{\rho_Gv_G}{a_t\mu_G}; \quad \mathrm{Pr}_G = \frac{c_P^G\mu_G}{\kappa_G} 
$$

where $\kappa_G$ is the thermal conductivity of the gas. All other parameters are constant.



# Model for Full-Scale Column

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy.integrate import solve_bvp

#Define Default Values of Scaled Parameters
vG0 = 1.0                          #m/s
vL0 = 0.03                         #m/s
at0 = 200.0                        #m2/m3
μL0 = 1e-3                         #Pa.s
L0 = 10.0                          #m

#Define Parameters
vG = vG0                           #m/s
vL = vL0                           #m/s
at = at0                           #m2/m3
R = 8.314                          #J/K.mol
T0 = 313.0                         #K
p0 = 1e5                           #Pa
MW_N2 = 0.028                      #kg/mol
ρG = p0/(R*T0)*MW_N2               #kg/m3
cpG = 1000.0                       #J/kg.K
ρL = 1000.0                        #kg/m3
cpL = 4100                         #J/kg.K
ΔH = -80000.0                      #J/mol
μG = 1.8e-5                        #Pa.s
μL = μL0                           #Pa.s
g = 9.81                           #m/s^2
σ = 72e-3                          #N/m
σc = 60e-3                         #N/m
κG = 2.5e-2                        #W/m.K

#Define BCs
cGinf = 10.0                       #mol/m3
cLinf = 0.1                        #mol/m3
TGinf = 313.0                      #K
TLinf = 313.0                      #K

#Define Domain
L = L0                             #m

#Calculate k
kOH = 10.0                         #m3/mol.s
cOH0 = 1e3                         #mol/m3
DCO2 = 2e-9                        #m2/s
kval = np.sqrt(kOH*cOH0*DCO2)         #m/s

#Calculate h
PrG = cpG*μG/κG
ReG = ρG*vG/(at*μG)
NuG = 5.23*ReG**0.7*PrG**(1/3)
h = NuG*κG*at/30.0                      #W/m2.K

#Calculate a
ReL = ρL*vL/(at*μL)
FrL = vL**2*at/g
WeL = ρL*vL**2/(σ*at)
a = at*(1-np.exp(-1.45 * (σc/σ)**0.75 * ReL**0.1 * FrL**(-0.05) * WeL**(0.2)))       #m2/m3

def DAC_fun(x, y):
    cG = y[0,:]
    cL = y[1,:]
    TG = y[2,:]
    TL = y[3,:]
    m = np.shape(x)[0]; n = np.shape(y)[0]
    dcGdz = -(kval*a*cG) / (vG) * np.ones(m)
    dcLdz = -(kval*a*cG) / (vL) * np.ones(m)
    dTGdz = (-h*a*(TG-TL)) / (vG*cpG*ρG) * np.ones(m)
    dTLdz = (-h*a*(TG-TL) + kval*a*cG*ΔH) / (vL*cpL*ρL)* np.ones(m)
    
    return np.vstack((dcGdz, dcLdz, dTGdz, dTLdz))

def DAC_bc(ya, yb):
    return np.array([ya[0] - cGinf, ya[2] - TGinf, yb[1] - cLinf, yb[3] - TLinf])
    
N = 10
x = np.linspace(0, L, N)
y0 = np.vstack((cGinf*np.ones(N), cLinf*np.ones(N), TGinf*np.ones(N), TLinf*np.ones(N)))

result = solve_bvp(DAC_fun, DAC_bc, x, y0, tol = 1e-3, max_nodes=1000)
print(result.message)

xplot = np.linspace(0,L,1000)
cGplot, cLplot, TGplot, TLplot = result.sol(xplot)
plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.plot(cLplot, xplot, label = "cL")
plt.legend()

plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.legend()

plt.figure()
plt.plot(TGplot, xplot, label = "TG")
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

plt.figure()
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

## Function to Calculate Scaling Factors


In [ ]:
import scipy.optimize as optimize


def DAC_Scale(ξ = 0.5, μ̄L = 1.0, atdash = 200.0, vLdash = 0.02, vGdash = 1.0, μLdash = 1e-3,
             g = 9.81, σc = 0.072, σ = 0.072, ρL = 1000.0):

    def resids_1(x):
        āt = x
        v̄G = ((1/āt)**0.3)**(1/0.7)
        v̄L = v̄G

        return v̄G / ((1 - np.exp(-1.45 * (σc/σ)**0.75 * \
        ((ρL*v̄L*vLdash)/(āt*atdash*μ̄L*μLdash))**0.1 * \
        (v̄L**2*vLdash**2*āt*atdash/g)**(-0.05) * \
        (ρL*v̄L**2*vLdash**2 / (σ*āt*atdash))**0.2))*āt*ξ) - \
        1 / ((1 - np.exp(-1.45 * (σc/σ)**0.75 * \
        ((ρL*vLdash)/(atdash*μLdash))**0.1 * \
        (vLdash**2*atdash/g)**(-0.05) * \
        (ρL*vLdash**2/(σ*atdash))**0.2)))


    sol = optimize.root_scalar(resids_1, bracket=[0.0001, 10000.0], method='brentq')
    āt = sol.root
    v̄G = ((1/āt)**0.3)**(1/0.7)
    v̄L = v̄G
    
    return (āt, v̄G, v̄L, μ̄L)

ξvals = np.linspace(0.1,10.0,100)
ātvals = [DAC_Scale(ξ = ξ, μ̄L = 1.0)[0] for ξ in ξvals]
v̄Gvals = [DAC_Scale(ξ = ξ, μ̄L = 1.0)[1] for ξ in ξvals]

ātvals_10μ = [DAC_Scale(ξ = ξ, μ̄L = 10.0)[0] for ξ in ξvals]
v̄Gvals_10μ = [DAC_Scale(ξ = ξ, μ̄L = 10.0)[1] for ξ in ξvals]

plt.figure()
plt.plot(1./ξvals, ātvals, 'C0', label = "Area (Const Liq. Visc.)")
plt.plot(1./ξvals, v̄Gvals, 'C1', label = "Gas Velocity (Const Liq. Visc.)")
plt.plot(1./ξvals, ātvals_10μ, 'C0--', label = "Area (10x Liq. Visc.)")
plt.plot(1./ξvals, v̄Gvals_10μ, 'C1--', label = "Gas Velocity (10x Liq. Visc.)")
plt.plot(1./ξvals, 1./ξvals, 'k', label = "y = x")
plt.legend()
plt.grid()
plt.ylim(0,10)
plt.xlim(0,10)
plt.xlabel("Decrease in Column Length")
plt.ylabel("Increase in Variable")
plt.savefig("ScaleFactor.png")
plt.show()


## Test Scaling for a 10x Smaller Column

In [ ]:
#Calculate Scaling Factors
ξ = 0.1
(āt, v̄G, v̄L, μ̄L) = DAC_Scale(ξ = ξ, μ̄L = 1.0, atdash = at0, vLdash = vL0, vGdash = vG0, μLdash = μL0,
             g = g, σc = σc, σ = σ, ρL = ρL)

#Define Parameters
vG = vG0*v̄G                        #m/s
vL = vL0*v̄L                        #m/s
at = at0*āt                        #m2/m3
R = 8.314                          #J/K.mol
T0 = 313.0                         #K
p0 = 1e5                           #Pa
MW_N2 = 0.028                      #kg/mol
ρG = p0/(R*T0)*MW_N2               #kg/m3
cpG = 1000.0                       #J/kg.K
ρL = 1000.0                        #kg/m3
cpL = 4100                         #J/kg.K
ΔH = -80000.0                      #J/mol
μG = 1.8e-5                        #Pa.s
μL = 1e-3*μ̄L                       #Pa.s
g = 9.81                           #m/s^2
σ = 72e-3                          #N/m
σc = 60e-3                         #N/m
κG = 2.5e-2                        #W/m.K

#Define BCs
cGinf = 10.0                       #mol/m3
cLinf = 0.1                        #mol/m3
TGinf = 313.0                      #K
TLinf = 313.0                      #K

#Define Domain
L = L0*ξ                           #m

#Calculate k
kOH = 10.0                         #m3/mol.s
cOH0 = 1e3                         #mol/m3
DCO2 = 2e-9                        #m2/s
kval = np.sqrt(kOH*cOH0*DCO2)         #m/s

#Calculate h
PrG = cpG*μG/κG
ReG = ρG*vG/(at*μG)
NuG = 5.23*ReG**0.7*PrG**(1/3)
h = NuG*κG*at/30                      #W/m2.K

#Calculate a
ReL = ρL*vL/(at*μL)
FrL = vL**2*at/g
WeL = ρL*vL**2/(σ*at)
a = at*(1-np.exp(-1.45 * (σc/σ)**0.75 * ReL**0.1 * FrL**(-0.05) * WeL**(0.2)))       #m2/m3

def DAC_fun(x, y):
    cG = y[0,:]
    cL = y[1,:]
    TG = y[2,:]
    TL = y[3,:]
    m = np.shape(x)[0]; n = np.shape(y)[0]
    dcGdz = -(kval*a*cG) / (vG) * np.ones(m)
    dcLdz = -(kval*a*cG) / (vL) * np.ones(m)
    dTGdz = (-h*a*(TG-TL)) / (vG*cpG*ρG) * np.ones(m)
    dTLdz = (-h*a*(TG-TL) + kval*a*cG*ΔH) / (vL*cpL*ρL)* np.ones(m)
    
    return np.vstack((dcGdz, dcLdz, dTGdz, dTLdz))

def DAC_bc(ya, yb):
    return np.array([ya[0] - cGinf, ya[2] - TGinf, yb[1] - cLinf, yb[3] - TLinf])
    
N = 10
x = np.linspace(0, L, N)
y0 = np.vstack((cGinf*np.ones(N), cLinf*np.ones(N), TGinf*np.ones(N), TLinf*np.ones(N)))

result_0p5 = solve_bvp(DAC_fun, DAC_bc, x, y0, tol = 1e-3, max_nodes=1000)
print(result_0p5.message)

xplot_0p5 = np.linspace(0,L,1000)
cGplot_0p5, cLplot_0p5, TGplot_0p5, TLplot_0p5 = result_0p5.sol(xplot_0p5)
plt.figure()
plt.plot(cGplot_0p5, xplot_0p5, label = "cG")
plt.plot(cLplot_0p5, xplot_0p5, label = "cL")
plt.legend()

plt.figure()
plt.plot(cGplot_0p5, xplot_0p5, label = "cG, 1m")
plt.plot(cGplot, xplot*ξ, 'C1--', label = "cG, 10m")
plt.xlabel("cG (mol/m3)")
plt.ylabel("z/L")
plt.legend()

plt.figure()
plt.plot(TGplot_0p5, xplot_0p5, label = "TG")
plt.plot(TLplot_0p5, xplot_0p5, label = "TL")
plt.legend()

plt.figure()
plt.plot(TGplot_0p5, xplot_0p5, label = "TL, 1m")
plt.plot(TGplot, xplot*ξ, 'C1--', label = "TL, 10m")
plt.xlabel("T (K)")
plt.ylabel("z/L")
plt.legend()

plt.figure()
plt.subplot(1,2,1)
plt.plot(TGplot_0p5, xplot_0p5, label = "TG, 1m")
plt.plot(TGplot, xplot*ξ, 'C1--', label = "TG, 10m")
plt.xlabel("T (K)")
plt.ylabel("z/L")
plt.legend()

plt.subplot(1,2,2)
plt.plot(cGplot_0p5, xplot_0p5, label = "cG, 1m")
plt.plot(cGplot, xplot*ξ, 'C1--', label = "cG, 10m")
plt.xlabel("cG (mol/m3)")

plt.legend()

plt.figure()
plt.subplot(1,2,1)
plt.plot(TGplot, xplot, label = "TG, 10m")
plt.xlabel("T (K)")
plt.ylabel("z")

plt.subplot(1,2,2)
plt.plot(TGplot_0p5, xplot_0p5, label = "TG, 1m")
plt.xlabel("T (K)")


plt.figure()
plt.subplot(1,2,1)
plt.plot(TGplot_0p5, xplot_0p5, label = "TG, 1m")
plt.plot(TGplot, xplot*ξ, 'C1--', label = "TG, 10m")
plt.xlabel("T (K)")
plt.ylabel("z/L")
plt.legend()

print("Average Temperature Deviation:", np.mean(np.abs(TGplot_0p5 - TGplot)), "K")


It is clear the analytical scaling predictions are confirmed computationally, with a deviation in the (dimensionless) profiles of less than 0.00001K. 

# MEA Model

Now we know scaling works for the physical solvent system, we now test it for a more sophisticated, MEA model, described by 5 ODEs:

$$    
v_G \frac{dc_G}{dz} = k(c_L, T_L)a(c_G - c_G^{*}(c_L, T_L))
$$

$$
v_L \frac{dc_L}{dz} = k(c_L, T_L)a(c_G - c_G^*(c_L, T_L))
$$

$$
    v_G \frac{dw_G}{dz} = k_GaRT_G(w_G - w_G^{*}(T_L))
$$

$$
    v_Gc_P^G\rho_G \frac{dT^G}{dz} = -ha(T^G-T^L) 
$$

$$
    v_Lc_P^L\rho_L \frac{dT^L}{dz} = -ha(T^G-T^L) + k(c_L, T_L)a(c_G - c_G^*(c_L,T_L))\Delta H + k_GaRT_G(w_G - w_G^*(T_L)) \Delta H_w
$$

We need to supply functional forms for the equilibrium concentration of water above the solvent, $w_G^{*}(T_L)$, the equilibrium concentration of CO2 above the solvent, $c_G^{*}(c_L, T_L)$, and the mass transfer coefficient, $c_G^{*}(T_L)$. For simplicity, we will supply a *linear* functional form for $k(c_L, T_L)$, as we believe the algebra for the scaling laws works in this case (though we suspect it will work generally.)

## $w_G^*(T_L)$

We have the following Antoine equation for water from the NIST Webbook (https://webbook.nist.gov/cgi/cbook.cgi?ID=C7732185&Mask=4&Type=ANTOINE&Plot=on):

$$
p_w\ \mathrm{[Pa]} = 10^5 \times 10^{4.6543 − (1435.264 / (T_L\ [\mathrm{K}] -64.848))}
$$

We can convert this to an equilibrium, gas-phase water concentration by assuming,

 - Rault's law holds for water in the amine solution.
 - The mole fraction of water is a constant and equal to $x_w = 0.9$ (this is not unreasonable).
 - The gas phase is ideal.

We then have

$$
w_G^*(T_L) = \frac{0.9 p_w(T_L)}{RT_L}
$$

We then have

$$
w_G^*(T_L) = \frac{0.9 \times 10^5 \times 10^{4.6543 − (1435.264 / (T_L\ [\mathrm{K}] -64.848))}}{{RT_L}}
$$



In [ ]:
def wGstar(T):
    return 0.9e5 * 10**(4.6543 - (1435.264 / (T - 64.848))) / (R * T)
    
plt.figure()
T = np.linspace(300,350, 100)
wG1 = wGstar(T)
#wG2 = wGstar_approx(T)
plt.plot(T, wG1, 'k')
plt.xlabel('T (K)')
plt.ylabel('wG* (mol/m3)')

## $c_G^*(T_L, c_L)$

We consider the function of Gabrielson et al. for the partial pressure of CO2 above an MEA solution:


$$
p_{CO2} = K(T, \theta) \left(\frac{a_0\theta}{a_0(1-2\theta)}\right)^2
$$

where $K$ is given by

$$
K = \exp \left(30.96 - \frac{10584}{T} - 7.187a_0\theta\right)
$$

Here, $a_0$ is the molar fraction of amines in the unloaded solvent; it is around $0.1$ for a 30wt% MEA solution. $\theta$ is the molar loading of CO2 in the amine. $c_L$, meanwhile, is the concentration of chemically-bound \ch{CO2} in solution. For a 30wt\% (around 5M) MEA solution, we can take $a_0 = 0.1$, and we can write $c_L = c_{MEA0}\times \theta \approx  5000 \ \mathrm{mol/m^3}\times \theta$, where $c_{MEA0}$ is the molar concentration of MEA in the unloaded solvent. This then gives

$$
p_{CO2} = K(T, \theta) \left(\frac{2\times 10^{-5} c_L}{0.1(1-4\times 10^{-4} c_L)}\right)^2
$$

$$
K = 10^3 \times \exp \left(30.96 - \frac{10584}{T} - \frac{0.7187}{5000}c_L\right)
$$

We now plot this for several temperatures.

In [ ]:
def K(TL,cL):
    return 1e3*np.exp(30.96 - 10584/TL - (0.7187/5000)*cL)

def pCO2(TL, cL):
    return K(TL, cL) * ((2e-5*cL)/(0.1*(1-4e-4*cL)))**2

N = 100
x = np.linspace(100, 2400, N)
y313 = [pCO2(313, x[i]) for i in range(N)]
y353 = [pCO2(353, x[i]) for i in range(N)]
y393 = [pCO2(393, x[i]) for i in range(N)]
plt.figure()
plt.semilogy(x/5000, y313, label = "313K")
plt.semilogy(x/5000, y353, label = "353K")
plt.semilogy(x/5000, y393, label = "393K")
plt.ylabel("CO2 Pressure (Pa)")
plt.xlabel("CO2 Loading")
plt.legend()
plt.grid()

This is in good agreement with Figure 1 of Gabrielson et al.

To convert this to a gas-phase concentration, we divide by RT,

$$
c_{G} = \frac{p_{CO2}}{RT} = \frac{K(T, \theta)}{RT} \left(\frac{2\times 10^{-5} c_L}{0.1(1-4\times 10^{-4} c_L)}\right)^2
$$

In [ ]:
def cGstar(TL, cL):
    R = 8.314
    return pCO2(TL, cL)/(R*TL)

N = 100
x = np.linspace(100, 2400, N)
y313 = [cGstar(313, x[i]) for i in range(N)]
y353 = [cGstar(353, x[i]) for i in range(N)]
y393 = [cGstar(393, x[i]) for i in range(N)]
plt.figure()
plt.semilogy(x/5000, y313, label = "313K")
plt.semilogy(x/5000, y353, label = "353K")
plt.semilogy(x/5000, y393, label = "393K")
plt.ylabel("CO2 Concentration (mol/m3)")
plt.xlabel("CO2 Loading")
plt.legend()
plt.grid()

## $k(c_L, T_L)$

This we will linearize. We start from the correlation of Versteeg et al. for the second order rate constant, $k_2$,

$$
k_2(T) \ [\mathrm{m^3/mol.s}] = 4.4\times 10^8 \exp \left(-5400/T\ \mathrm{[K]}\right)
$$

We are interested in $k$, defined as

$$
J = k\Delta c_G
$$

where $J$ is the flux of CO2 from the gas to the liquid, and $\Delta c_G$ is the driving force for CO2 transfer based on the gas-side concentration. Based on the \emph{liquid}-side concentration, we may estimate the flux from the following expression, valid because absorption of CO2 into MEA is governed by absorption with pseudo-first-order reaction,

$$
J = \sqrt{k_2 c_{MEA} D_{CO2}} \Delta c_{L}
$$

To convert $c_L$ to $c_G$, we need a Henry's constant for MEA:

$$
J = \sqrt{k_2 c_{MEA} D_{CO2}} \mathcal{H}\Delta c_{G}
$$

So that $k$ is equal to

$$
k = \sqrt{k_2 c_{MEA} D_{CO2}} \mathcal{H}
$$



For the Henry's constant, we use the correlation of Ma'mum and Svendsen is given as follows. First, they give a correlation for the Henry's constant of N2O in MEA:

$$
\ln H = C_1 + C_2/T + C_3 \ln T + C_4T
$$

$$
C_1 = -4052.21; C_2 = 110950; C_3 =  704.824; C_4 = -1.097938 
$$

Meanwhile, the ratio of the Henry's constant of CO2 in water and N2O in water is given by:

$$
\frac{H_{CO2 in water}}{H_{N2O in water}} = \exp \left(C_5 + C_6/T + C_7\ln T + C_8 T\right)
$$

where

$$
C_5 = -118.7848; C_6 = 2580.65; C_7 = 21.6287; C_8 = -0.044901,
$$

According to the N2O analogy, we can estimate $H_{CO2 in MEA}$ via

$$
H_{CO2 in MEA} = H_{N2O in MEA} \times \frac{H_{CO2 in water}}{H_{N2O in water}}
$$

This gives us:

$$
H_{CO2 in MEA} = \exp \left(C_1 + C_2/T + C_3 \ln T + C_4T \right) \times \exp \left(C_5 + C_6/T + C_7\ln T + C_8 T\right)
$$

$$
H_{CO2 in MEA} = \exp \left([C_1 + C_5] + [C_2 + C_6]/T + [C_3 + C_7] \ln T + [C_4 + C_8]T \right)
$$

Now, the only issue here is that this $H$ has units Pa/(mol/m3), and is thus defined via

$$
p_{i} = H \times c_{i,L}
$$

We instead want $\mathcal{H}$  defined by:

$$
c_{i,L} = \mathcal{H} \times c_{i,G}
$$

For this, we have,

$$
c_{i,L} = \mathcal{H} \times c_{i,G} = \mathcal{H} \times \frac{p_i}{RT} = \mathcal{H} \times \frac{ H \times c_{i,L}}{RT}
$$

So

$$
\mathcal{H} =  \frac{RT}{ H}
$$

So, overall, \mathcal{H} is dimensionless and is given by,


$$
\mathcal{H} = \mathcal{H}_{CO2 in MEA} = \frac{RT}{\exp \left([C_1 + C_5] + [C_2 + C_6]/T + [C_3 + C_7] \ln T + [C_4 + C_8]T \right)}
$$

We will write,

$$
\mathcal{H} = \frac{RT}{\exp \left(E_1 + E_2/T + E_3\ln T + E_4 T \right)}
$$

Where $E_1 = C_1 + C_5$, etc.

Overall, then, our correlation for $k$ is

$$
k = \sqrt{k_2 c_{MEA} D_{CO2}} \mathcal{H}
$$

$$
k = \sqrt{4.4\times 10^8 \exp \left(-5400/T\ \mathrm{[K]}\right) c_{MEA} D_{CO2}} \frac{RT}{\exp \left(E_1 + E_2/T + E_3\ln T + E_4 T \right)}
$$

Now, we will treat $D_{CO2}$ as a constant and equal to $1.9\times 10^{-9}\mathrm{\ m^2/s}$. The concentration of MEA relates to the loading as,

$$
c_{MEA} = c_{MEA0}(1-2\theta) = 5000 (1-2\theta)
$$

We already saw that $\theta = c_L/c_{MEA0}$, where for 30wt% MEA, $c_{MEA0} = 5000 mol/m3$, so we have

$$
c_{MEA} = c_{MEA0}(1-2c_L-c_{MEA0}) = c_{MEA0} - 2c_L
$$

Overall, then

$$
k(T, c_L) = \sqrt{4.4\times 10^8 \exp \left(-5400/T\ \mathrm{[K]}\right) (c_{MEA0} - 2c_L) D_{CO2}} \frac{RT}{\exp \left(E_1 + E_2/T + E_3\ln T + E_4 T \right)}
$$

In [ ]:
#Henry's constant, CO2 in 30wt% MEA, dimensionless (H = cL/cG)
def H_MEA(T):
    R = 8.314
    C1 = -4052.21; C2 = 110950; C3 =  704.824; C4 = -1.097938 
    C5 = -118.7848; C6 = 2580.65; C7 = 21.6287; C8 = -0.044901
    E1 = C1 + C5; E2 = C2 + C6; E3 = C3 + C7; E4 = C4 + C8
    return (R*T)/np.exp(E1 + E2/T + E3*np.log(T) + E4*T)


#Second order rate constant, mol/m3.s
def k2(T):
    return 4.4e8 * np.exp(-5400/T)

#Rate constant, J = k*ΔcG
def k(T, cL):
    cMEA0 = 5000.0     #mol/m3
    DCO2 = 1.9e-9      #m2/s
    return np.sqrt(k2(T)*(cMEA0 - 2*cL)*DCO2)*H_MEA(T)

plt.figure()

plt.plot(np.linspace(500, 2000, 100), [k(313, cL) for cL in np.linspace(500, 2000, 100)], color = "C1", label = "313K")
plt.plot(np.linspace(500, 2000, 100), [k(323, cL) for cL in np.linspace(500, 2000, 100)], color = "C0", label = "323K")
plt.plot(np.linspace(500, 2000, 100), [k(333, cL) for cL in np.linspace(500, 2000, 100)], color = "C2", label = "333K")
plt.plot(np.linspace(500, 2000, 100), [k(353, cL) for cL in np.linspace(500, 2000, 100)], color = "C4", label = "353K")

plt.xlabel("cL (mol/m3)")
plt.ylabel("k (m/s)")
plt.legend()



There error in $k(T, cL)$ is not to obad over the practical operating range (40C-60C).

# MEA Model

Now we have developed the necessary correlations, we are in a position to solve the 5 ODE's. 

$$    
v_G \frac{dc_G}{dz} = -k(c_L, T_L)a(c_G - c_G^{*}(c_L, T_L))
$$

$$
v_L \frac{dc_L}{dz} = -k(c_L, T_L)a(c_G - c_G^*(c_L, T_L))
$$

$$
    v_G \frac{dw_G}{dz} = -k_GaRT_G(w_G - w_G^{*}(T_L))
$$

$$
    v_Gc_P^G\rho_G \frac{dT^G}{dz} = -ha(T^G-T^L) 
$$

$$
    v_Lc_P^L\rho_L \frac{dT^L}{dz} = -ha(T^G-T^L) + k(c_L, T_L)a(c_G - c_G^*(c_L,T_L))\Delta H + k_GaRT_G(w_G - w_G^*(T_L)) \Delta H_w
$$

In [ ]:
#Define Default Values of Scaled Parameters
vG0 = 1.0                          #m/s
vL0 = 0.02                         #m/s
at0 = 200.0                        #m2/m3
μL0 = 1e-3                         #Pa.s
L0 = 10.0                          #m

#Define Parameters
vG = vG0                           #m/s
vL = vL0                           #m/s
at = at0                           #m2/m3
R = 8.314                          #J/K.mol
T0 = 313.0                         #K
p0 = 1e5                           #Pa
MW_N2 = 0.028                      #kg/mol
ρG = p0/(R*T0)*MW_N2               #kg/m3
cpG = 1000.0                       #J/kg.K
ρL = 1000.0                        #kg/m3
cpL = 4100                         #J/kg.K
ΔH = -80000.0                      #J/mol
ΔHw = -40000.0                     #J/mol
μG = 1.8e-5                        #Pa.s
μL = μL0                           #Pa.s
g = 9.81                           #m/s^2
σ = 72e-3                          #N/m
σc = 60e-3                         #N/m
κG = 2.5e-2                        #W/m.K
DG = 1.6e-5                        #m2/s

#Define BCs
cGinf = 0.3*1e5/(R*313.0)          #mol/m3
cLinf = 1500.0                     #mol/m3
wGinf = 0.05*1e5/(R*313.0)         #mol/m3
TGinf = 313.0                      #K
TLinf = 313.0                      #K

#Define Domain
L = L0                             #m

#Calculate k
#kOH = 10.0                         #m3/mol.s
#cOH0 = 1e3                         #mol/m3
#DCO2 = 2e-9                        #m2/s
#k = np.sqrt(kOH*cOH0*DCO2)         #m/s

#Calculate h and kG
PrG = cpG*μG/κG
ScG = μG/(ρG*DG)
ReG = ρG*vG/(at*μG)
NuG = 5.23*ReG**0.7*PrG**(1/3)
ShG = 5.23*ReG**0.7*ScG**(1/3)
h = NuG*κG*at/30                       #W/m2.K
kGRTG = ShG*DG*at/30                   #m/s

#Calculate a
ReL = ρL*vL/(at*μL)
FrL = vL**2*at/g
WeL = ρL*vL**2/(σ*at)
a = at*(1-np.exp(-1.45 * (σc/σ)**0.75 * ReL**0.1 * FrL**(-0.05) * WeL**(0.2)))       #m2/m3



def MEACol_fun(x, y):
    cG = y[0,:]
    cL = y[1,:]
    wG = y[2,:]
    TG = y[3,:]
    TL = y[4,:]
    m = np.shape(x)[0]; n = np.shape(y)[0]
    dcGdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vG) * np.ones(m)
    dcLdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vL) * np.ones(m)
    dwGdz = -(kGRTG*a*(wG - wGstar(TL))) / (vG) * np.ones(m)
    dTGdz = (-h*a*(TG-TL)) / (vG*cpG*ρG) * np.ones(m)
    dTLdz = (-h*a*(TG-TL) + k(TL, cL)*a*cG*ΔH + kGRTG*a*(wG - wGstar(TL))*ΔHw) / (vL*cpL*ρL)* np.ones(m)
    
    return np.vstack((dcGdz, dcLdz, dwGdz, dTGdz, dTLdz))

def MEACol_bc(ya, yb):
    return np.array([ya[0] - cGinf, ya[2] - wGinf, ya[3] - TGinf, yb[1] - cLinf, yb[4] - TLinf])
    
N = 10
x = np.linspace(0, L, N)
y0 = np.vstack((cGinf*np.ones(N), cLinf*np.ones(N), wGinf*np.ones(N), TGinf*np.ones(N), TLinf*np.ones(N)))

result = solve_bvp(MEACol_fun, MEACol_bc, x, y0, tol = 1e-3, max_nodes=1000)
print(result.message)

xplot = np.linspace(0,L,1000)
cGplot, cLplot, wGplot, TGplot, TLplot = result.sol(xplot)
plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.plot(cLplot, xplot, label = "cL")
plt.legend()

plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.legend()

plt.figure()
plt.plot(TGplot, xplot, label = "TG")
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

plt.figure()
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

# MEA Model 10x Scaled

In [ ]:
ξ = 0.1
(āt, v̄G, v̄L, μ̄L) = DAC_Scale(ξ = ξ, μ̄L = 1.0, atdash = at0, vLdash = vL0, vGdash = vG0, μLdash = μL0,
             g = g, σc = σc, σ = σ, ρL = ρL)

#Define Parameters
vG = vG0*v̄G                           #m/s
vL = vL0*v̄L                           #m/s
at = at0*āt                           #m2/m3
R = 8.314                          #J/K.mol
T0 = 313.0                         #K
p0 = 1e5                           #Pa
MW_N2 = 0.028                      #kg/mol
ρG = p0/(R*T0)*MW_N2               #kg/m3
cpG = 1000.0                       #J/kg.K
ρL = 1000.0                        #kg/m3
cpL = 4100                         #J/kg.K
ΔH = -80000.0                      #J/mol
ΔHw = -40000.0                     #J/mol
μG = 1.8e-5                        #Pa.s
μL = μL0*μ̄L                           #Pa.s
g = 9.81                           #m/s^2
σ = 72e-3                          #N/m
σc = 60e-3                         #N/m
κG = 2.5e-2                        #W/m.K
DG = 1.6e-5                        #m2/s

#Define BCs
cGinf = 0.3*1e5/(R*313.0)          #mol/m3
cLinf = 1500.0                     #mol/m3
wGinf = 0.05*1e5/(R*313.0)         #mol/m3
TGinf = 313.0                      #K
TLinf = 313.0                      #K

#Define Domain
L = L0*ξ                             #m

#Calculate k
#kOH = 10.0                         #m3/mol.s
#cOH0 = 1e3                         #mol/m3
#DCO2 = 2e-9                        #m2/s
#k = np.sqrt(kOH*cOH0*DCO2)         #m/s

#Calculate h and kG
PrG = cpG*μG/κG
ScG = μG/(ρG*DG)
ReG = ρG*vG/(at*μG)
NuG = 5.23*ReG**0.7*PrG**(1/3)
ShG = 5.23*ReG**0.7*ScG**(1/3)
h = NuG*κG*at/30                       #W/m2.K
kGRTG = ShG*DG*at/30                   #m/s

#Calculate a
ReL = ρL*vL/(at*μL)
FrL = vL**2*at/g
WeL = ρL*vL**2/(σ*at)
a = at*(1-np.exp(-1.45 * (σc/σ)**0.75 * ReL**0.1 * FrL**(-0.05) * WeL**(0.2)))       #m2/m3



def MEACol_fun(x, y):
    cG = y[0,:]
    cL = y[1,:]
    wG = y[2,:]
    TG = y[3,:]
    TL = y[4,:]
    m = np.shape(x)[0]; n = np.shape(y)[0]
    dcGdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vG) * np.ones(m)
    dcLdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vL) * np.ones(m)
    dwGdz = -(kGRTG*a*(wG - wGstar(TL))) / (vG) * np.ones(m)
    dTGdz = (-h*a*(TG-TL)) / (vG*cpG*ρG) * np.ones(m)
    dTLdz = (-h*a*(TG-TL) + k(TL, cL)*a*cG*ΔH + kGRTG*a*(wG - wGstar(TL))*ΔHw) / (vL*cpL*ρL)* np.ones(m)
    
    return np.vstack((dcGdz, dcLdz, dwGdz, dTGdz, dTLdz))

def MEACol_bc(ya, yb):
    return np.array([ya[0] - cGinf, ya[2] - wGinf, ya[3] - TGinf, yb[1] - cLinf, yb[4] - TLinf])
    
N = 10
x = np.linspace(0, L, N)
y0 = np.vstack((cGinf*np.ones(N), cLinf*np.ones(N), wGinf*np.ones(N), TGinf*np.ones(N), TLinf*np.ones(N)))

result = solve_bvp(MEACol_fun, MEACol_bc, x, y0, tol = 1e-3, max_nodes=1000)
print(result.message)

xplotscaled = np.linspace(0,L,1000)
cGplotscaled, cLplotscaled, wGplotscaled, TGplotscaled, TLplotscaled = result.sol(xplotscaled)
plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.plot(cLplot, xplot, label = "cL")
plt.legend()

plt.figure()
plt.plot(cGplot, xplot, label = "cG")
plt.legend()

plt.figure()
plt.plot(TGplot, xplot, label = "TG")
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

plt.figure()
plt.plot(TLplot, xplot, label = "TL")
plt.legend()

plt.figure()
plt.plot(TGplot, xplot*ξ, label = "Full Scale", ls = "-")
plt.plot(TGplotscaled, xplotscaled, label = "10x Smaller", ls = "--")
plt.xlabel("TG (K)")
plt.ylabel("z/L")
plt.legend()

plt.figure()
plt.plot(cGplot, xplot*ξ, ls = "-", label = "Full Scale")
plt.plot(cGplotscaled, xplotscaled, ls = "--", label = "10x Smaller")
plt.xlabel("cG (mol/m3)")
plt.ylabel("z/L")
plt.legend()

# Figure 2 of Manuscript:  Plot of MEA Scaling

In [ ]:
plt.figure(dpi = 300)
plt.plot(TGplot, xplot*ξ, label = "Full Size", ls = "-", color = "#D95319")
plt.plot(TGplotscaled, xplotscaled, label = "Scaled Model", lw = 2, ls = "--", color = "#0072BD")
#plt.xlabel("TG (K)")
#plt.ylabel("z/L")
fig = plt.gcf()
fig.set_size_inches(2.5,4)
plt.legend()
plt.savefig("figc.pdf")

plt.figure(dpi = 300)
#plt.plot(TGplot, xplot*ξ, label = "Full Size", ls = "-", color = "#0072BD")
plt.plot(TGplotscaled, xplotscaled, label = "Scaled Model", ls = "--", color = "#0072BD")
#plt.xlabel("TG (K)")
#plt.ylabel("z")
fig = plt.gcf()
fig.set_size_inches(1.5,1.3)
plt.savefig("figa.pdf")

plt.figure(dpi = 300)
#plt.plot(TGplot, xplot*ξ, label = "Full Size", ls = "-", color = "#0072BD")
plt.plot(TGplot, xplot, label = "Full Size", ls = "-", color = "#D95319")
#plt.xlabel("TG (K)")
#plt.ylabel("z")
fig = plt.gcf()
fig.set_size_inches(1.5,4)

plt.savefig("figb.pdf")


# Comparison with Validated Julia Model & Pilot Data

We have a Julia model, which was previous published in 10.1002/aic.17243, which has been well-validated against pilot data. We now compare how see how this simple model compares against it. We begin by putting all of the above code into a single function.

In [ ]:
def MEA_Absorber_fun(
    vG = 50.0/(1e5/(8.314*313.0)),      #m/s
    vL = (2.0*173)/45000,               #m/s
    at = 200.0,                         #m2/m3
    R = 8.314,                          #J/K.mol
    T0 = 313.0,                         #K
    p0 = 1e5,                           #Pa
    MW_N2 = 0.028,                      #kg/mol
    ρG = 1e5/(8.314*313.0)*0.028,       #kg/m3
    cpG = 1000.0,                       #J/kg.K
    ρL = 1000.0,                        #kg/m3
    cpL = 3500,                         #J/kg.K
    ΔH = -82000.0,                      #J/mol
    ΔHw = -40000.0,                     #J/mol
    μG = 1.8e-5,                        #Pa.s
    μL = 1e-3,                          #Pa.s
    g = 9.81,                           #m/s^2
    σ = 72e-3,                          #N/m
    σc = 60e-3,                         #N/m
    κG = 2.5e-2,                        #W/m.K
    DG = 1.6e-5,                        #m2/s
    L = 10.0,                           #m
    N = 10,
    cGinf = 0.1*1e5/(8.314*313.0),      #mol/m3
    cLinf = 5000.0*0.25,                #mol/m3
    wGinf = 0.05*1e5/(8.314*313.0),     #mol/m3
    TGinf = 313.0,                      #K
    TLinf = 313.0,                      #K
    max_nodes = 1000, 
    xw = 0.9,
    Prefactor = 5.32, 
    cMEA0 = 5000.0,                     #mol/m3
    DCO2 = 1.9e-9):                     #m2/s
    
    #Calculate h and kG from Onda correlation
    PrG = cpG*μG/κG
    ScG = μG/(ρG*DG)
    ReG = ρG*vG/(at*μG)
    NuG = Prefactor*ReG**0.7*PrG**(1/3)*(3.4)**(-2)
    ShG = Prefactor*ReG**0.7*ScG**(1/3)*(3.4)**(-2)
    h = NuG*κG*at                                #W/m2.K
    kGRTG = ShG*DG*at                            #m/s
 
    #Calculate aw from Onda correlation
    ReL = ρL*vL/(at*μL)
    FrL = vL**2*at/g
    WeL = ρL*vL**2/(σ*at)
    a = at*(1-np.exp(-1.45 * (σc/σ)**0.75 * ReL**0.1 * FrL**(-0.05) * WeL**(0.2)))       #m2/m3

    #Water vapor pressure above solvent, Pa
    def wGstar(T, xw = xw):
        return xw*1e5*10**(4.6543 - (1435.264 / (T - 64.848))) / (R * T)
    
    #VLE for MEA, from Gabrielson et al.
    def K(TL,cL, cMEA0 = cMEA0):
        return 1e3*np.exp(30.96 - 10584/TL - (0.7187/cMEA0)*cL)

    def pCO2(TL, cL, cMEA0 = cMEA0, xw = xw):
        return K(TL, cL) * (((1-xw)*cL/cMEA0)/((1-xw)*(1-2*cL/cMEA0)))**2
    
    def cGstar(TL, cL, cMEA0 = cMEA0, xw = xw):
        R = 8.314
        return pCO2(TL, cL)/(R*TL)

    #Henry's constant, CO2 in 30wt% MEA, dimensionless (H = cL/cG)
    def H_MEA(T):
        R = 8.314
        C1 = -4052.21; C2 = 110950; C3 =  704.824; C4 = -1.097938 
        C5 = -118.7848; C6 = 2580.65; C7 = 21.6287; C8 = -0.044901
        E1 = C1 + C5; E2 = C2 + C6; E3 = C3 + C7; E4 = C4 + C8
        return (R*T)/np.exp(E1 + E2/T + E3*np.log(T) + E4*T)

    #Second order rate constant, mol/m3.s
    def k2(T):
        return 4.4e8 * np.exp(-5400/T)

    #Rate constant, J = k*ΔcG
    def k(T, cL, cMEA0 = cMEA0, DCO2 = DCO2):
        return np.sqrt(k2(T)*(cMEA0 - 2*cL)*DCO2)*H_MEA(T)

    def MEACol_fun(x, y):
        cG = y[0,:]
        cL = y[1,:]
        wG = y[2,:]
        TG = y[3,:]
        TL = y[4,:]
        m = np.shape(x)[0]; n = np.shape(y)[0]
        dcGdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vG) * np.ones(m)
        dcLdz = -(k(TL, cL)*a*(cG - cGstar(TL, cL))) / (vL) * np.ones(m)
        dwGdz = -(kGRTG*a*(wG - wGstar(TL))) / (vG) * np.ones(m)
        dTGdz = (-h*a*(TG-TL)) / (vG*cpG*ρG) * np.ones(m)
        dTLdz = (-h*a*(TG-TL) + k(TL, cL)*a*cG*ΔH + kGRTG*a*(wG - wGstar(TL))*ΔHw) / (vL*cpL*ρL)* np.ones(m)

        return np.vstack((dcGdz, dcLdz, dwGdz, dTGdz, dTLdz))

    def MEACol_bc(ya, yb):
        return np.array([ya[0] - cGinf, ya[2] - wGinf, ya[3] - TGinf, yb[1] - cLinf, yb[4] - TLinf])

    x = np.linspace(0, L, N)
    y0 = np.vstack((cGinf*np.ones(N), cLinf*np.ones(N), wGinf*np.ones(N), TGinf*np.ones(N), TLinf*np.ones(N)))

    result = solve_bvp(MEACol_fun, MEACol_bc, x, y0, tol = 1e-3, max_nodes=max_nodes)
    print(result.message)

    xplot = np.linspace(0,L,1000)
    cGplot, cLplot, wGplot, TGplot, TLplot = result.sol(xplot)
    
    return (result, xplot, cGplot, cLplot, wGplot, TGplot, TLplot)

Now we solve this and compare with the results from Figure 5b from the AIChE paper, 10.1002/aic.17243.

In [ ]:
result = MEA_Absorber_fun(
    vG = 50.0/(1e5/(8.314*313.0)),      #m/s
    vL = (2.0*173)/45000,               #m/s
    at = 200.0,                         #m2/m3
    R = 8.314,                          #J/K.mol
    T0 = 313.0,                         #K
    p0 = 1e5,                           #Pa
    MW_N2 = 0.028,                      #kg/mol
    ρG = 1e5/(8.314*313.0)*0.028,       #kg/m3
    cpG = 1000.0,                       #J/kg.K
    ρL = 1000.0,                        #kg/m3
    cpL = 3500,                         #J/kg.K
    ΔH = -82000.0,                      #J/mol
    ΔHw = -40000.0,                     #J/mol
    μG = 1.8e-5,                        #Pa.s
    μL = 1e-3,                          #Pa.s
    g = 9.81,                           #m/s^2
    σ = 72e-3,                          #N/m
    σc = 60e-3,                         #N/m
    κG = 2.5e-2,                        #W/m.K
    DG = 1.6e-5,                        #m2/s
    L = 6.394,                           #m
    N = 10,
    cGinf = 0.1*1e5/(8.314*313.0),      #mol/m3
    cLinf = 5000.0*0.25,                #mol/m3
    wGinf = 0.05*1e5/(8.314*313.0),     #mol/m3
    TGinf = 313.0,                      #K
    TLinf = 313.0,                      #K
    max_nodes = 10000, 
    xw = 0.9,
    cMEA0 = 5000.0,                     #mol/m3
    DCO2 = 1.9e-9)                     #m2/s
    

z_Julia  = np.array([0.0, 0.001001001001001001, 0.002002002002002002, 0.003003003003003003, 0.004004004004004004, 0.005005005005005005, 0.006006006006006006, 0.007007007007007007, 0.008008008008008008, 0.009009009009009009, 0.01001001001001001, 0.011011011011011011, 0.012012012012012012, 0.013013013013013013, 0.014014014014014014, 0.015015015015015015, 0.016016016016016016, 0.01701701701701702, 0.018018018018018018, 0.01901901901901902, 0.02002002002002002, 0.021021021021021023, 0.022022022022022022, 0.023023023023023025, 0.024024024024024024, 0.025025025025025027, 0.026026026026026026, 0.02702702702702703, 0.028028028028028028, 0.02902902902902903, 0.03003003003003003, 0.031031031031031032, 0.03203203203203203, 0.03303303303303303, 0.03403403403403404, 0.035035035035035036, 0.036036036036036036, 0.037037037037037035, 0.03803803803803804, 0.03903903903903904, 0.04004004004004004, 0.04104104104104104, 0.042042042042042045, 0.043043043043043044, 0.044044044044044044, 0.04504504504504504, 0.04604604604604605, 0.04704704704704705, 0.04804804804804805, 0.04904904904904905, 0.05005005005005005, 0.05105105105105105, 0.05205205205205205, 0.05305305305305305, 0.05405405405405406, 0.055055055055055056, 0.056056056056056056, 0.057057057057057055, 0.05805805805805806, 0.05905905905905906, 0.06006006006006006, 0.06106106106106106, 0.062062062062062065, 0.06306306306306306, 0.06406406406406406, 0.06506506506506507, 0.06606606606606606, 0.06706706706706707, 0.06806806806806807, 0.06906906906906907, 0.07007007007007007, 0.07107107107107107, 0.07207207207207207, 0.07307307307307308, 0.07407407407407407, 0.07507507507507508, 0.07607607607607608, 0.07707707707707707, 0.07807807807807808, 0.07907907907907907, 0.08008008008008008, 0.08108108108108109, 0.08208208208208208, 0.08308308308308308, 0.08408408408408409, 0.08508508508508508, 0.08608608608608609, 0.08708708708708708, 0.08808808808808809, 0.0890890890890891, 0.09009009009009009, 0.09109109109109109, 0.0920920920920921, 0.09309309309309309, 0.0940940940940941, 0.09509509509509509, 0.0960960960960961, 0.0970970970970971, 0.0980980980980981, 0.0990990990990991, 0.1001001001001001, 0.1011011011011011, 0.1021021021021021, 0.1031031031031031, 0.1041041041041041, 0.10510510510510511, 0.1061061061061061, 0.10710710710710711, 0.10810810810810811, 0.1091091091091091, 0.11011011011011011, 0.1111111111111111, 0.11211211211211211, 0.11311311311311312, 0.11411411411411411, 0.11511511511511512, 0.11611611611611612, 0.11711711711711711, 0.11811811811811812, 0.11911911911911911, 0.12012012012012012, 0.12112112112112113, 0.12212212212212212, 0.12312312312312312, 0.12412412412412413, 0.12512512512512514, 0.12612612612612611, 0.12712712712712712, 0.12812812812812813, 0.12912912912912913, 0.13013013013013014, 0.13113113113113112, 0.13213213213213212, 0.13313313313313313, 0.13413413413413414, 0.13513513513513514, 0.13613613613613615, 0.13713713713713713, 0.13813813813813813, 0.13913913913913914, 0.14014014014014015, 0.14114114114114115, 0.14214214214214213, 0.14314314314314314, 0.14414414414414414, 0.14514514514514515, 0.14614614614614616, 0.14714714714714713, 0.14814814814814814, 0.14914914914914915, 0.15015015015015015, 0.15115115115115116, 0.15215215215215216, 0.15315315315315314, 0.15415415415415415, 0.15515515515515516, 0.15615615615615616, 0.15715715715715717, 0.15815815815815815, 0.15915915915915915, 0.16016016016016016, 0.16116116116116116, 0.16216216216216217, 0.16316316316316315, 0.16416416416416416, 0.16516516516516516, 0.16616616616616617, 0.16716716716716717, 0.16816816816816818, 0.16916916916916916, 0.17017017017017017, 0.17117117117117117, 0.17217217217217218, 0.17317317317317318, 0.17417417417417416, 0.17517517517517517, 0.17617617617617617, 0.17717717717717718, 0.1781781781781782, 0.17917917917917917, 0.18018018018018017, 0.18118118118118118, 0.18218218218218218, 0.1831831831831832, 0.1841841841841842, 0.18518518518518517, 0.18618618618618618, 0.1871871871871872, 0.1881881881881882, 0.1891891891891892, 0.19019019019019018, 0.19119119119119118, 0.1921921921921922, 0.1931931931931932, 0.1941941941941942, 0.19519519519519518, 0.1961961961961962, 0.1971971971971972, 0.1981981981981982, 0.1991991991991992, 0.2002002002002002, 0.2012012012012012, 0.2022022022022022, 0.2032032032032032, 0.2042042042042042, 0.20520520520520522, 0.2062062062062062, 0.2072072072072072, 0.2082082082082082, 0.2092092092092092, 0.21021021021021022, 0.21121121121121122, 0.2122122122122122, 0.2132132132132132, 0.21421421421421422, 0.21521521521521522, 0.21621621621621623, 0.2172172172172172, 0.2182182182182182, 0.21921921921921922, 0.22022022022022023, 0.22122122122122123, 0.2222222222222222, 0.22322322322322322, 0.22422422422422422, 0.22522522522522523, 0.22622622622622623, 0.22722722722722724, 0.22822822822822822, 0.22922922922922923, 0.23023023023023023, 0.23123123123123124, 0.23223223223223224, 0.23323323323323322, 0.23423423423423423, 0.23523523523523523, 0.23623623623623624, 0.23723723723723725, 0.23823823823823823, 0.23923923923923923, 0.24024024024024024, 0.24124124124124124, 0.24224224224224225, 0.24324324324324326, 0.24424424424424424, 0.24524524524524524, 0.24624624624624625, 0.24724724724724725, 0.24824824824824826, 0.24924924924924924, 0.2502502502502503, 0.25125125125125125, 0.25225225225225223, 0.25325325325325326, 0.25425425425425424, 0.2552552552552553, 0.25625625625625625, 0.25725725725725723, 0.25825825825825827, 0.25925925925925924, 0.2602602602602603, 0.26126126126126126, 0.26226226226226224, 0.26326326326326327, 0.26426426426426425, 0.2652652652652653, 0.26626626626626626, 0.2672672672672673, 0.2682682682682683, 0.26926926926926925, 0.2702702702702703, 0.27127127127127126, 0.2722722722722723, 0.2732732732732733, 0.27427427427427425, 0.2752752752752753, 0.27627627627627627, 0.2772772772772773, 0.2782782782782783, 0.27927927927927926, 0.2802802802802803, 0.28128128128128127, 0.2822822822822823, 0.2832832832832833, 0.28428428428428426, 0.2852852852852853, 0.2862862862862863, 0.2872872872872873, 0.2882882882882883, 0.28928928928928926, 0.2902902902902903, 0.2912912912912913, 0.2922922922922923, 0.2932932932932933, 0.29429429429429427, 0.2952952952952953, 0.2962962962962963, 0.2972972972972973, 0.2982982982982983, 0.2992992992992993, 0.3003003003003003, 0.3013013013013013, 0.3023023023023023, 0.3033033033033033, 0.30430430430430433, 0.3053053053053053, 0.3063063063063063, 0.3073073073073073, 0.3083083083083083, 0.30930930930930933, 0.3103103103103103, 0.3113113113113113, 0.3123123123123123, 0.3133133133133133, 0.31431431431431434, 0.3153153153153153, 0.3163163163163163, 0.3173173173173173, 0.3183183183183183, 0.31931931931931934, 0.3203203203203203, 0.3213213213213213, 0.32232232232232233, 0.3233233233233233, 0.32432432432432434, 0.3253253253253253, 0.3263263263263263, 0.32732732732732733, 0.3283283283283283, 0.32932932932932935, 0.3303303303303303, 0.33133133133133136, 0.33233233233233234, 0.3333333333333333, 0.33433433433433435, 0.3353353353353353, 0.33633633633633636, 0.33733733733733734, 0.3383383383383383, 0.33933933933933935, 0.34034034034034033, 0.34134134134134136, 0.34234234234234234, 0.3433433433433433, 0.34434434434434436, 0.34534534534534533, 0.34634634634634637, 0.34734734734734735, 0.3483483483483483, 0.34934934934934936, 0.35035035035035034, 0.35135135135135137, 0.35235235235235235, 0.3533533533533533, 0.35435435435435436, 0.35535535535535534, 0.3563563563563564, 0.35735735735735735, 0.35835835835835833, 0.35935935935935936, 0.36036036036036034, 0.3613613613613614, 0.36236236236236236, 0.3633633633633634, 0.36436436436436437, 0.36536536536536535, 0.3663663663663664, 0.36736736736736736, 0.3683683683683684, 0.36936936936936937, 0.37037037037037035, 0.3713713713713714, 0.37237237237237236, 0.3733733733733734, 0.3743743743743744, 0.37537537537537535, 0.3763763763763764, 0.37737737737737737, 0.3783783783783784, 0.3793793793793794, 0.38038038038038036, 0.3813813813813814, 0.38238238238238237, 0.3833833833833834, 0.3843843843843844, 0.38538538538538536, 0.3863863863863864, 0.38738738738738737, 0.3883883883883884, 0.3893893893893894, 0.39039039039039036, 0.3913913913913914, 0.3923923923923924, 0.3933933933933934, 0.3943943943943944, 0.3953953953953954, 0.3963963963963964, 0.3973973973973974, 0.3983983983983984, 0.3993993993993994, 0.4004004004004004, 0.4014014014014014, 0.4024024024024024, 0.4034034034034034, 0.4044044044044044, 0.40540540540540543, 0.4064064064064064, 0.4074074074074074, 0.4084084084084084, 0.4094094094094094, 0.41041041041041043, 0.4114114114114114, 0.4124124124124124, 0.4134134134134134, 0.4144144144144144, 0.41541541541541543, 0.4164164164164164, 0.4174174174174174, 0.4184184184184184, 0.4194194194194194, 0.42042042042042044, 0.4214214214214214, 0.42242242242242245, 0.42342342342342343, 0.4244244244244244, 0.42542542542542544, 0.4264264264264264, 0.42742742742742745, 0.42842842842842843, 0.4294294294294294, 0.43043043043043044, 0.4314314314314314, 0.43243243243243246, 0.43343343343343343, 0.4344344344344344, 0.43543543543543545, 0.4364364364364364, 0.43743743743743746, 0.43843843843843844, 0.4394394394394394, 0.44044044044044045, 0.44144144144144143, 0.44244244244244246, 0.44344344344344344, 0.4444444444444444, 0.44544544544544545, 0.44644644644644643, 0.44744744744744747, 0.44844844844844844, 0.4494494494494494, 0.45045045045045046, 0.45145145145145144, 0.45245245245245247, 0.45345345345345345, 0.4544544544544545, 0.45545545545545546, 0.45645645645645644, 0.4574574574574575, 0.45845845845845845, 0.4594594594594595, 0.46046046046046046, 0.46146146146146144, 0.4624624624624625, 0.46346346346346345, 0.4644644644644645, 0.46546546546546547, 0.46646646646646645, 0.4674674674674675, 0.46846846846846846, 0.4694694694694695, 0.47047047047047047, 0.47147147147147145, 0.4724724724724725, 0.47347347347347346, 0.4744744744744745, 0.4754754754754755, 0.47647647647647645, 0.4774774774774775, 0.47847847847847846, 0.4794794794794795, 0.4804804804804805, 0.48148148148148145, 0.4824824824824825, 0.48348348348348347, 0.4844844844844845, 0.4854854854854855, 0.4864864864864865, 0.4874874874874875, 0.48848848848848847, 0.4894894894894895, 0.4904904904904905, 0.4914914914914915, 0.4924924924924925, 0.4934934934934935, 0.4944944944944945, 0.4954954954954955, 0.4964964964964965, 0.4974974974974975, 0.4984984984984985, 0.4994994994994995, 0.5005005005005005, 0.5015015015015015, 0.5025025025025025, 0.5035035035035035, 0.5045045045045045, 0.5055055055055055, 0.5065065065065065, 0.5075075075075075, 0.5085085085085085, 0.5095095095095095, 0.5105105105105106, 0.5115115115115115, 0.5125125125125125, 0.5135135135135135, 0.5145145145145145, 0.5155155155155156, 0.5165165165165165, 0.5175175175175175, 0.5185185185185185, 0.5195195195195195, 0.5205205205205206, 0.5215215215215215, 0.5225225225225225, 0.5235235235235235, 0.5245245245245245, 0.5255255255255256, 0.5265265265265265, 0.5275275275275275, 0.5285285285285285, 0.5295295295295295, 0.5305305305305306, 0.5315315315315315, 0.5325325325325325, 0.5335335335335335, 0.5345345345345346, 0.5355355355355356, 0.5365365365365365, 0.5375375375375375, 0.5385385385385385, 0.5395395395395396, 0.5405405405405406, 0.5415415415415415, 0.5425425425425425, 0.5435435435435435, 0.5445445445445446, 0.5455455455455456, 0.5465465465465466, 0.5475475475475475, 0.5485485485485485, 0.5495495495495496, 0.5505505505505506, 0.5515515515515516, 0.5525525525525525, 0.5535535535535535, 0.5545545545545546, 0.5555555555555556, 0.5565565565565566, 0.5575575575575575, 0.5585585585585585, 0.5595595595595596, 0.5605605605605606, 0.5615615615615616, 0.5625625625625625, 0.5635635635635635, 0.5645645645645646, 0.5655655655655656, 0.5665665665665666, 0.5675675675675675, 0.5685685685685685, 0.5695695695695696, 0.5705705705705706, 0.5715715715715716, 0.5725725725725725, 0.5735735735735735, 0.5745745745745746, 0.5755755755755756, 0.5765765765765766, 0.5775775775775776, 0.5785785785785785, 0.5795795795795796, 0.5805805805805806, 0.5815815815815816, 0.5825825825825826, 0.5835835835835835, 0.5845845845845846, 0.5855855855855856, 0.5865865865865866, 0.5875875875875876, 0.5885885885885885, 0.5895895895895896, 0.5905905905905906, 0.5915915915915916, 0.5925925925925926, 0.5935935935935935, 0.5945945945945946, 0.5955955955955956, 0.5965965965965966, 0.5975975975975976, 0.5985985985985987, 0.5995995995995996, 0.6006006006006006, 0.6016016016016016, 0.6026026026026026, 0.6036036036036037, 0.6046046046046046, 0.6056056056056056, 0.6066066066066066, 0.6076076076076076, 0.6086086086086087, 0.6096096096096096, 0.6106106106106106, 0.6116116116116116, 0.6126126126126126, 0.6136136136136137, 0.6146146146146146, 0.6156156156156156, 0.6166166166166166, 0.6176176176176176, 0.6186186186186187, 0.6196196196196196, 0.6206206206206206, 0.6216216216216216, 0.6226226226226226, 0.6236236236236237, 0.6246246246246246, 0.6256256256256256, 0.6266266266266266, 0.6276276276276276, 0.6286286286286287, 0.6296296296296297, 0.6306306306306306, 0.6316316316316316, 0.6326326326326326, 0.6336336336336337, 0.6346346346346347, 0.6356356356356356, 0.6366366366366366, 0.6376376376376376, 0.6386386386386387, 0.6396396396396397, 0.6406406406406406, 0.6416416416416416, 0.6426426426426426, 0.6436436436436437, 0.6446446446446447, 0.6456456456456456, 0.6466466466466466, 0.6476476476476476, 0.6486486486486487, 0.6496496496496497, 0.6506506506506506, 0.6516516516516516, 0.6526526526526526, 0.6536536536536537, 0.6546546546546547, 0.6556556556556556, 0.6566566566566566, 0.6576576576576577, 0.6586586586586587, 0.6596596596596597, 0.6606606606606606, 0.6616616616616616, 0.6626626626626627, 0.6636636636636637, 0.6646646646646647, 0.6656656656656657, 0.6666666666666666, 0.6676676676676677, 0.6686686686686687, 0.6696696696696697, 0.6706706706706707, 0.6716716716716716, 0.6726726726726727, 0.6736736736736737, 0.6746746746746747, 0.6756756756756757, 0.6766766766766766, 0.6776776776776777, 0.6786786786786787, 0.6796796796796797, 0.6806806806806807, 0.6816816816816816, 0.6826826826826827, 0.6836836836836837, 0.6846846846846847, 0.6856856856856857, 0.6866866866866866, 0.6876876876876877, 0.6886886886886887, 0.6896896896896897, 0.6906906906906907, 0.6916916916916916, 0.6926926926926927, 0.6936936936936937, 0.6946946946946947, 0.6956956956956957, 0.6966966966966966, 0.6976976976976977, 0.6986986986986987, 0.6996996996996997, 0.7007007007007007, 0.7017017017017017, 0.7027027027027027, 0.7037037037037037, 0.7047047047047047, 0.7057057057057057, 0.7067067067067067, 0.7077077077077077, 0.7087087087087087, 0.7097097097097097, 0.7107107107107107, 0.7117117117117117, 0.7127127127127127, 0.7137137137137137, 0.7147147147147147, 0.7157157157157157, 0.7167167167167167, 0.7177177177177178, 0.7187187187187187, 0.7197197197197197, 0.7207207207207207, 0.7217217217217218, 0.7227227227227228, 0.7237237237237237, 0.7247247247247247, 0.7257257257257257, 0.7267267267267268, 0.7277277277277278, 0.7287287287287287, 0.7297297297297297, 0.7307307307307307, 0.7317317317317318, 0.7327327327327328, 0.7337337337337337, 0.7347347347347347, 0.7357357357357357, 0.7367367367367368, 0.7377377377377378, 0.7387387387387387, 0.7397397397397397, 0.7407407407407407, 0.7417417417417418, 0.7427427427427428, 0.7437437437437437, 0.7447447447447447, 0.7457457457457457, 0.7467467467467468, 0.7477477477477478, 0.7487487487487487, 0.7497497497497497, 0.7507507507507507, 0.7517517517517518, 0.7527527527527528, 0.7537537537537538, 0.7547547547547547, 0.7557557557557557, 0.7567567567567568, 0.7577577577577578, 0.7587587587587588, 0.7597597597597597, 0.7607607607607607, 0.7617617617617618, 0.7627627627627628, 0.7637637637637638, 0.7647647647647647, 0.7657657657657657, 0.7667667667667668, 0.7677677677677678, 0.7687687687687688, 0.7697697697697697, 0.7707707707707707, 0.7717717717717718, 0.7727727727727728, 0.7737737737737738, 0.7747747747747747, 0.7757757757757757, 0.7767767767767768, 0.7777777777777778, 0.7787787787787788, 0.7797797797797797, 0.7807807807807807, 0.7817817817817818, 0.7827827827827828, 0.7837837837837838, 0.7847847847847848, 0.7857857857857858, 0.7867867867867868, 0.7877877877877878, 0.7887887887887888, 0.7897897897897898, 0.7907907907907908, 0.7917917917917918, 0.7927927927927928, 0.7937937937937938, 0.7947947947947948, 0.7957957957957958, 0.7967967967967968, 0.7977977977977978, 0.7987987987987988, 0.7997997997997998, 0.8008008008008008, 0.8018018018018018, 0.8028028028028028, 0.8038038038038038, 0.8048048048048048, 0.8058058058058059, 0.8068068068068068, 0.8078078078078078, 0.8088088088088088, 0.8098098098098098, 0.8108108108108109, 0.8118118118118118, 0.8128128128128128, 0.8138138138138138, 0.8148148148148148, 0.8158158158158159, 0.8168168168168168, 0.8178178178178178, 0.8188188188188188, 0.8198198198198198, 0.8208208208208209, 0.8218218218218218, 0.8228228228228228, 0.8238238238238238, 0.8248248248248248, 0.8258258258258259, 0.8268268268268268, 0.8278278278278278, 0.8288288288288288, 0.8298298298298298, 0.8308308308308309, 0.8318318318318318, 0.8328328328328328, 0.8338338338338338, 0.8348348348348348, 0.8358358358358359, 0.8368368368368369, 0.8378378378378378, 0.8388388388388388, 0.8398398398398398, 0.8408408408408409, 0.8418418418418419, 0.8428428428428428, 0.8438438438438438, 0.8448448448448449, 0.8458458458458459, 0.8468468468468469, 0.8478478478478478, 0.8488488488488488, 0.8498498498498499, 0.8508508508508509, 0.8518518518518519, 0.8528528528528528, 0.8538538538538538, 0.8548548548548549, 0.8558558558558559, 0.8568568568568569, 0.8578578578578578, 0.8588588588588588, 0.8598598598598599, 0.8608608608608609, 0.8618618618618619, 0.8628628628628628, 0.8638638638638638, 0.8648648648648649, 0.8658658658658659, 0.8668668668668669, 0.8678678678678678, 0.8688688688688688, 0.8698698698698699, 0.8708708708708709, 0.8718718718718719, 0.8728728728728729, 0.8738738738738738, 0.8748748748748749, 0.8758758758758759, 0.8768768768768769, 0.8778778778778779, 0.8788788788788788, 0.8798798798798799, 0.8808808808808809, 0.8818818818818819, 0.8828828828828829, 0.8838838838838838, 0.8848848848848849, 0.8858858858858859, 0.8868868868868869, 0.8878878878878879, 0.8888888888888888, 0.8898898898898899, 0.8908908908908909, 0.8918918918918919, 0.8928928928928929, 0.8938938938938938, 0.8948948948948949, 0.8958958958958959, 0.8968968968968969, 0.8978978978978979, 0.8988988988988988, 0.8998998998998999, 0.9009009009009009, 0.9019019019019019, 0.9029029029029029, 0.9039039039039038, 0.9049049049049049, 0.9059059059059059, 0.9069069069069069, 0.9079079079079079, 0.908908908908909, 0.9099099099099099, 0.9109109109109109, 0.9119119119119119, 0.9129129129129129, 0.913913913913914, 0.914914914914915, 0.9159159159159159, 0.9169169169169169, 0.9179179179179179, 0.918918918918919, 0.91991991991992, 0.9209209209209209, 0.9219219219219219, 0.9229229229229229, 0.923923923923924, 0.924924924924925, 0.9259259259259259, 0.9269269269269269, 0.9279279279279279, 0.928928928928929, 0.92992992992993, 0.9309309309309309, 0.9319319319319319, 0.9329329329329329, 0.933933933933934, 0.934934934934935, 0.9359359359359359, 0.9369369369369369, 0.9379379379379379, 0.938938938938939, 0.93993993993994, 0.9409409409409409, 0.9419419419419419, 0.9429429429429429, 0.943943943943944, 0.944944944944945, 0.9459459459459459, 0.9469469469469469, 0.9479479479479479, 0.948948948948949, 0.94994994994995, 0.950950950950951, 0.9519519519519519, 0.9529529529529529, 0.953953953953954, 0.954954954954955, 0.955955955955956, 0.9569569569569569, 0.9579579579579579, 0.958958958958959, 0.95995995995996, 0.960960960960961, 0.9619619619619619, 0.9629629629629629, 0.963963963963964, 0.964964964964965, 0.965965965965966, 0.9669669669669669, 0.9679679679679679, 0.968968968968969, 0.96996996996997, 0.970970970970971, 0.9719719719719719, 0.972972972972973, 0.973973973973974, 0.974974974974975, 0.975975975975976, 0.9769769769769769, 0.977977977977978, 0.978978978978979, 0.97997997997998, 0.980980980980981, 0.9819819819819819, 0.982982982982983, 0.983983983983984, 0.984984984984985, 0.985985985985986, 0.986986986986987, 0.987987987987988, 0.988988988988989, 0.98998998998999, 0.990990990990991, 0.991991991991992, 0.992992992992993, 0.993993993993994, 0.994994994994995, 0.995995995995996, 0.996996996996997, 0.997997997997998, 0.998998998998999, 1.0])
TG_Julia = np.array([313.0, 313.421199018293, 313.8337117038718, 314.2377356068692, 314.63346388298385, 315.02108534481147, 315.40078452179415, 315.7727417164209, 316.13713307309746, 316.4941306382603, 316.84390243785015, 317.18661253663623, 317.52242112154204, 317.8514845646452, 318.1739555001969, 318.4899829037993, 318.79971215124, 319.103285111955, 319.40084020958165, 319.6925124908413, 319.972770818844, 320.25318073615966, 320.5280910306025, 320.7976240633187, 321.06189917660566, 321.32103277108183, 321.5751383647956, 321.82432667261236, 322.0687056588618, 322.3083806093607, 322.54345419658694, 322.77402652918977, 323.00019523355127, 323.2220554990499, 323.43970013203773, 323.6532196351711, 323.8627022454175, 324.0682339835686, 324.2698987354912, 324.467778284107, 324.65810467851094, 324.8487228157522, 325.03578750304206, 325.2193726719964, 325.3995503964663, 325.5763909446339, 325.7499628247435, 325.92033282112874, 326.0875660528404, 326.2517260005251, 326.4128745495537, 326.5710720416507, 326.72637729991544, 326.8788476660784, 327.02853905646026, 327.17550598090486, 327.3198015716693, 327.46147763909033, 327.6005846974078, 327.73717198637775, 327.86862911261835, 328.0003677224541, 328.12972609752217, 328.256748749256, 328.38147907068304, 328.5039593649512, 328.62423087866796, 328.7423338202701, 328.8583073903812, 328.9721898132209, 329.08401835031003, 329.1938293251139, 329.30165815895674, 329.407539384302, 329.51150665904544, 329.61359280327366, 329.7138298201391, 329.8122489070492, 329.9088804692999, 330.00375415855575, 330.0950527537472, 330.1865304491785, 330.2763342957747, 330.364491095409, 330.45102697689623, 330.53596741061773, 330.61933723017546, 330.70116064693605, 330.7814612580529, 330.8602620742016, 330.93758552918564, 331.0134534896238, 331.08788726961063, 331.16090765439145, 331.23253490606476, 331.3027887725728, 331.3716885022148, 331.43925286788016, 331.50550016897, 331.5704482420841, 331.63285359351, 331.6952800570388, 331.7564578184861, 331.81640299244077, 331.8751312874034, 331.9326580135934, 331.9889980963207, 332.0441660861197, 332.0981761637381, 332.1510421512773, 332.2027775271023, 332.2533954282336, 332.30290865759946, 332.3513296942093, 332.39867070760795, 332.4449435598289, 332.4901598125533, 332.5343307305225, 332.5774673018486, 332.61958024054445, 332.65986785698755, 332.6999845525873, 332.73910796113194, 332.77724776330626, 332.8144133977868, 332.85061406597964, 332.88585874006003, 332.9201561703643, 332.95351488963826, 332.98594321557977, 333.01744926407656, 333.0480409520748, 333.0777260007619, 333.1065119396601, 333.13440611568984, 333.1614157007905, 333.1875476923901, 333.2128089197979, 333.2372060431699, 333.2607455685377, 333.28298838762925, 333.3048484746654, 333.3258694613911, 333.34605727014815, 333.3654176876536, 333.3839563683664, 333.4016788394341, 333.41859050657143, 333.4346966564869, 333.4500024591796, 333.464512974173, 333.478233156794, 333.4911678583176, 333.5033218305863, 333.5146997268948, 333.525306110861, 333.5351454601379, 333.5442221664138, 333.55254054014245, 333.5601048108119, 333.5667901697657, 333.57287351205616, 333.578214933481, 333.5828183893604, 333.58668777338823, 333.58982692032004, 333.5922396089208, 333.593929567188, 333.59490047330866, 333.59515595912643, 333.5946996104203, 333.5935349745243, 333.5916655623199, 333.5890948482366, 333.58582627401563, 333.5818632501532, 333.5772091595182, 333.5718673635952, 333.56584120249704, 333.5591339949652, 333.5519033614947, 333.5438574043891, 333.53514019653704, 333.5257550029791, 333.51570508088986, 333.50499368166635, 333.4936240530197, 333.48159944268355, 333.46892309914125, 333.45559827368953, 333.4416282219699, 333.42701620762364, 333.41176550442776, 333.3958793962953, 333.3793611796096, 333.3622141652319, 333.34444168073236, 333.32604707339146, 333.30703371020195, 333.28740497826135, 333.2675750777197, 333.24673800256505, 333.2252957982632, 333.20325195457525, 333.18060999038516, 333.1573734549123, 333.1335459293523, 333.1091310285405, 333.0841324015761, 333.0585537331359, 333.03239874449105, 333.0056711955874, 332.97837488526517, 332.95051365184906, 332.9220913748021, 332.8931119753539, 332.86357941805977, 332.83349771085045, 332.80287090558085, 332.77170309923525, 332.7406377616816, 332.70841103950585, 332.6756557985361, 332.64237632099656, 332.60857693696516, 332.5742620245756, 332.5394360104097, 332.5041033699643, 332.4682686278782, 332.43193635797, 332.3951111832257, 332.357797776279, 332.32000085944964, 332.28172520462283, 332.2429756327754, 332.20375701410677, 332.1640742681162, 332.12393236359037, 332.08333631794034, 332.0422911964242, 332.0016362112818, 331.95971705036186, 331.91736419126454, 331.8745828873247, 331.83137843811755, 331.78775618907224, 331.74372152968937, 331.69927989341954, 331.6544367572993, 331.60919764096946, 331.5635681051478, 331.5175537509521, 331.47116021981617, 331.4243931928933, 331.3772583880497, 331.32976155952343, 331.2819084971702, 331.23370502640137, 331.18515700638284, 331.13627032719506, 331.0880385200029, 331.0384987914614, 330.9886381351521, 330.93846255596895, 330.88797808470844, 330.83719077800316, 330.7861067147765, 330.73473199408875, 330.68307273533696, 330.6311350772999, 330.5789251752179, 330.5264491985367, 330.47371333040394, 330.42072376764975, 330.3674867191193, 330.3140083999861, 330.2602950324639, 330.20635284473013, 330.1521880708786, 330.0978069482721, 330.0443095558531, 329.98951845669734, 329.9345295880396, 329.87934917648477, 329.8239834428529, 329.7684386021796, 329.71272086363456, 329.6568364229055, 329.60079145981194, 329.54459214144646, 329.48824462037106, 329.431755032095, 329.3751294898068, 329.31837408510444, 329.2614948882213, 329.2044979480264, 329.14738928932206, 329.0901749037618, 329.0328607538791, 328.9754527723753, 328.919107603913, 328.8615312217705, 328.8038784982073, 328.74615522876064, 328.6883671721288, 328.6305200501712, 328.57261954454896, 328.5146712902996, 328.4566808800647, 328.39865386302006, 328.34059574423986, 328.2825119775899, 328.2244079667565, 328.1662890671006, 328.10816058565734, 328.05002778052915, 327.99189585197223, 327.93376994227015, 327.87565513964313, 327.8175564781487, 327.76064024955565, 327.70258818336265, 327.64456691931935, 327.58658125644945, 327.5286359346528, 327.4707356342661, 327.41288497062976, 327.3550884948217, 327.2973506955605, 327.2396759991172, 327.1820687651763, 327.1245332846303, 327.06707378390314, 327.00969442517334, 326.95239930586064, 326.8951924517921, 326.83807781809776, 326.78105929364943, 326.7241407011467, 326.6673257970143, 326.6117513405906, 326.5551525354862, 326.4986681885122, 326.44230177500566, 326.3860567024404, 326.3299363095576, 326.2739438624088, 326.21808255760953, 326.16235552343323, 326.10676581862015, 326.05131642892536, 325.99601027190846, 325.9408501977592, 325.88583898858093, 325.83097935355727, 325.7762739325433, 325.72172529898245, 325.6673359599633, 325.6131083542831, 325.559044848147, 325.50622403884734, 325.4524921775203, 325.3989310797586, 325.34554284876907, 325.29232952399576, 325.2392930802311, 325.1864354252854, 325.1337584037513, 325.08126379797363, 325.0289533260445, 324.97682864218524, 324.92489134076766, 324.8731429564349, 324.8215849619013, 324.7702187675201, 324.71904572612516, 324.66806713351474, 324.6172842279974, 324.5666981865303, 324.51631013038957, 324.467122949952, 324.41713000417155, 324.36733806289953, 324.31774803027344, 324.26836075919766, 324.21917705062896, 324.17019765219084, 324.1214232617977, 324.07285452861043, 324.0244920512102, 323.9763363794094, 323.92838801734996, 323.880647423457, 323.83311500838136, 323.7857911368102, 323.7386761309092, 323.6917702705661, 323.6450737919946, 323.59858688706277, 323.5523097085393, 323.5071616603647, 323.46130003958604, 323.4156483674293, 323.3702066411703, 323.3249748225607, 323.2799528373067, 323.23514057403276, 323.19053788749943, 323.14614459953737, 323.1019604976217, 323.0579853365997, 323.01421884134555, 322.9706607067187, 322.92731059596946, 322.8841681425655, 322.84123295297405, 322.7985046068707, 322.75598265585853, 322.7136666235569, 322.6715560097719, 322.6304864007021, 322.58878094751293, 322.5472792794409, 322.5059808014766, 322.4648848979065, 322.4239909319494, 322.3832982447358, 322.3428061579814, 322.30251397487064, 322.26242097903776, 322.222526435535, 322.18282959329446, 322.14332968515413, 322.1040259266375, 322.06491751692073, 322.0260036413851, 321.9872834718243, 321.9487561656815, 321.91042086537254, 321.87227670187065, 321.83508001448206, 321.79731169489617, 321.7597318542728, 321.7223395782261, 321.6851339434005, 321.6481140172297, 321.61127885682663, 321.57462751104214, 321.53815902129315, 321.5018724209229, 321.4657667352567, 321.4298409838828, 321.39409418077, 321.358525333433, 321.32313344281295, 321.28791750563073, 321.2528765146635, 321.2180094585317, 321.1833153201685, 321.1487930795476, 321.11512707382843, 321.08094217350686, 321.04692611639496, 321.01307787204564, 320.9793964091003, 320.94588069514344, 320.9125296955408, 320.87934237486695, 320.8463176976948, 320.8134546282828, 320.7807521298951, 320.748209166763, 320.7158247043181, 320.6835977087587, 320.65152714610133, 320.619611984045, 320.587851192472, 320.5562437434442, 320.5247886098465, 320.49348476633423, 320.4629527962873, 320.4319454957142, 320.40108644773204, 320.3703746380275, 320.3398090561532, 320.3093886954527, 320.27911255193976, 320.24897962512813, 320.21898891881136, 320.1891394409734, 320.15943020273767, 320.129860219811, 320.10042851287335, 320.07113410742323, 320.04197603261593, 320.0129533222042, 319.9840650153664, 319.9553101567297, 319.9266877957396, 319.8981969858996, 319.8704027165302, 319.8421696092166, 319.81406526677375, 319.78608876452813, 319.75823918397174, 319.7305156127323, 319.70291714358433, 319.6754428747506, 319.6480919107052, 319.6208633621583, 319.59375634505324, 319.56676998131314, 319.5399033994203, 319.5131557343901, 319.486526126899, 319.46001372317215, 319.4336176760382, 319.40733714498884, 319.38117129606496, 319.35511930062665, 319.32969801301806, 319.3038690281168, 319.27815146583777, 319.252544522689, 319.2270474020016, 319.2016593139228, 319.1763794746161, 319.1512071061259, 319.1261414372229, 319.1011817033571, 319.0763271459587, 319.05157701248106, 319.0269305571344, 319.002387040898, 318.97794573110485, 318.9536059005777, 318.92936682860284, 318.9052278011266, 318.8811881107565, 318.85724705610795, 318.8338798481795, 318.81013204566847, 318.7864808246501, 318.76292550784905, 318.7394654245158, 318.7160999104264, 318.6928283072897, 318.6696499622751, 318.646564228893, 318.6235704669136, 318.6006680419595, 318.57785632507245, 318.5551346934695, 318.5325025305977, 318.5099592260128, 318.4875041743956, 318.46513677598915, 318.44285643705365, 318.4206625698635, 318.3985545925711, 318.3769715490883, 318.3550319367214, 318.3331765116027, 318.31140471312875, 318.28971598645893, 318.26810978251604, 318.2465855575727, 318.2251427725713, 318.2037808939914, 318.1824993937601, 318.16129774906966, 318.1401754416921, 318.1191319586434, 318.09816679230687, 318.07727944041784, 318.0564694053816, 318.0357361940177, 318.01507931823545, 317.9944982950601, 317.9739926466286, 317.95396978354887, 317.93361198440033, 317.9133281625962, 317.8931178581971, 317.87298061611, 317.8529159860884, 317.8329235224478, 317.8130027833062, 317.7931533313761, 317.77337473390276, 317.7536665626183, 317.7340283930018, 317.7144598047528, 317.6949603820098, 317.6755297133501, 317.6561673914777, 317.63687301248694, 317.6176461765409, 317.5984864879953, 317.57939355539764, 317.56074687470726, 317.54178497878104, 317.52288869444294, 317.50405764489267, 317.4852914572805, 317.4665897627074, 317.44795219602986, 317.42937839510046, 317.41086800145393, 317.39242066029686, 317.3740360205093, 317.3557137340126, 317.3374534559738, 317.3192548451377, 317.3011175638273, 317.28304127784304, 317.26502565568126, 317.24707036887537, 317.2291750923207, 317.21133950427037, 317.1939182309028, 317.1761998892188, 317.15854029524394, 317.1409391389265, 317.1233961133745, 317.1059109148553, 317.08848324266285, 317.07111279839876, 317.05379928654685, 317.03654241451835, 317.0193418926573, 317.0021974337749, 316.98510875309097, 316.9680755686604, 316.95109760138007, 316.93417457496986, 316.9173062153949, 316.90049225075694, 316.88373241180113, 316.8670264319288, 316.85070657135435, 316.8341064560765, 316.81755941985324, 316.80106520529165, 316.7846235575006, 316.7682342240907, 316.7518969550832, 316.73561150225277, 316.7193776195939, 316.7031950634163, 316.68706359234267, 316.6709829669991, 316.654952949755, 316.63897330519774, 316.6230438001586, 316.60716420371244, 316.59133428685305, 316.5755538220308, 316.5598225837185, 316.54414034848156, 316.5288190874084, 316.5132332273157, 316.4976957159787, 316.4822063371672, 316.46676487662637, 316.45137112207686, 316.4360248631525, 316.4207258908107, 316.4054739977011, 316.3902689783024, 316.3751106289092, 316.35999874745255, 316.3449331331041, 316.3299135867532, 316.31493991106197, 316.3000119104654, 316.2851293910308, 316.2702921598571, 316.2555000255155, 316.24075279823126, 316.22634390301204, 316.2116850385073, 316.19707052474223, 316.1825001775871, 316.1679738144771, 316.1534912544119, 316.1390523179135, 316.12465682650384, 316.11030460298747, 316.0959954716203, 316.08172925808685, 316.06750578941956, 316.0533248935295, 316.03918639964434, 316.02509013840074, 316.0110359418447, 315.9970236433819, 315.9830530772367, 315.96912407862857, 315.9552364840908, 315.9416666553954, 315.9278605640995, 315.9140953971747, 315.90037099561016, 315.8866872016439, 315.873043858763, 315.85944081167486, 315.84587790584834, 315.8323549877223, 315.81887190489755, 315.8054285061071, 315.7920246412033, 315.7786601606713, 315.765334915991, 315.7520487597741, 315.73880154576386, 315.7255931288239, 315.71242336453884, 315.69929210911516, 315.6861992197985, 315.67340527450295, 315.66038793339055, 315.64740853904954, 315.63446695228544, 315.62156303491207, 315.60869664975166, 315.59586766061557, 315.583075931904, 315.5703213287507, 315.5576037172312, 315.54492296432977, 315.5322789379637, 315.5196715065307, 315.50710053916276, 315.49456590591296, 315.48206747775555, 315.46960512658484, 315.4571787249703, 315.444788145844, 315.43243326294237, 315.4203599893206, 315.40807541614276, 315.3958261679642, 315.3836121214111, 315.371433153935, 315.3592891438127, 315.34717997013365, 315.3351055124527, 315.3230656508802, 315.3110602663007, 315.299089240341, 315.28715245540405, 315.27524979429234, 315.2633811403299, 315.2515463776015, 315.23974539095286, 315.22797806599044, 315.2162442889577, 315.2045439463166, 315.1928769251206, 315.1814754646517, 315.16987408980026, 315.1583057040652, 315.14677019688867, 315.1352674583986, 315.12379737940796, 315.11235985140763, 315.10095476626583, 315.0895820162732, 315.0782414943658, 315.0669330940963, 315.05565670966695, 315.0444122356372, 315.03319956692195, 315.0220185990714, 315.0108692282738, 314.9997513513563, 314.9886648657304, 314.9776096689844, 314.9665856590985, 314.955812289161, 314.94484973089635, 314.93391805911676, 314.92301717381434, 314.91214697555927, 314.9013073654996, 314.8904982453566, 314.8797195171672, 314.8689710832901, 314.8582528466302, 314.8475647106131, 314.83690657921267, 314.8262783567386, 314.8156799477313, 314.80511125726434, 314.7945721909552, 314.7840626549652, 314.7735825559816, 314.76313180088357, 314.7527102967734, 314.7425255130128, 314.73216165399907, 314.7218267722841, 314.711520776689, 314.7012435765291, 314.69099508161435, 314.68077520224614, 314.67058384899764, 314.6604209326882, 314.6502863646057, 314.64018005648376, 314.630101920524, 314.6200518692531, 314.6100298153376, 314.6000356718924, 314.59006935250227, 314.5801307712217, 314.57021984257165, 314.560336481305, 314.55048060227057, 314.54084842241826, 314.53104670882436, 314.52127222642235, 314.5115248915411, 314.50180462093806, 314.4921113317991, 314.48244494173724, 314.472805368606, 314.46319253044754, 314.45360634571085, 314.4440467332321, 314.43451361225004, 314.425006902323, 314.4155265230865, 314.4060723945512, 314.3966444371387, 314.3872425716817, 314.37786671942365, 314.3685168018805, 314.35919274058637, 314.3500801709482, 314.3408070750097, 314.3315596037938, 314.3223376801097, 314.31314122714235, 314.3039701684525, 314.29482442797615, 314.2857039298669, 314.27660859842393, 314.2675383583028, 314.2584931344992, 314.2494728523592, 314.2404774375422, 314.23150681574697, 314.2225609129815, 314.21363965561744, 314.20474297038976, 314.195870784397, 314.1870230250326, 314.17819961968644, 314.1695762409642, 314.1608008434627, 314.15204958549555, 314.14332239554074, 314.13461920240894, 314.12593993524376, 314.11728452352145, 314.1086528969198, 314.1000449852286, 314.0914607185523, 314.0829000272982, 314.0743628421805, 314.06584909421844, 314.057358714453, 314.04889163417414, 314.0404477849962, 314.03202709885807, 314.0236295080229, 314.0152549450501, 314.00690334251976, 313.9987409834176, 313.99043464439774, 313.98215106640146, 313.9738901829237, 313.9656519277568, 313.9574362349907, 313.94924303901263, 313.9410722743987, 313.932923875812, 313.92479777819517, 313.9166939167623, 313.90861222700016, 313.9005526446863, 313.8925151056212, 313.88449954579715, 313.8765059014981, 313.8685341092999, 313.86058410607046, 313.8526558289608, 313.84474921519035, 313.83702169101224, 313.8291577858491, 313.8213153580651, 313.8134943456405, 313.8056946868234, 313.7979163201296, 313.7901591843426, 313.7824232184248, 313.7747083614064, 313.7670145525673, 313.75934173143247, 313.75168983777235, 313.7440588116275, 313.73644859307865, 313.72885912234307, 313.7212903399022, 313.713742186502, 313.70621460315226, 313.6987075311264, 313.691220911818, 313.68390381163147, 313.6764575159599, 313.66903149918, 313.6616257033163, 313.65424007063604, 313.6468745436488, 313.6395290651066, 313.6322035779318, 313.62489802510083, 313.6176123498134, 313.610346495492, 313.60310040578236, 313.59587402457595, 313.5886672958311, 313.5814801635919, 313.5743125721421, 313.56716446600615, 313.56003578994915, 313.5529264889769, 313.54583650825793, 313.5389070187836, 313.53185513039995, 313.52482239946903, 313.51780877168846, 313.5108141929767, 313.50383860947295, 313.49688196753715, 313.4899442136915, 313.48302529450297, 313.4761251567381, 313.4692437473672, 313.4623810135646, 313.45553690272754, 313.44871136234565, 313.44190433995374, 313.43511578330254, 313.42834564036264, 313.421593859325, 313.4148603886006, 313.40814517678547, 313.4015819344343, 313.394902723555, 313.3882416188679, 313.38159856942667, 313.37497352448696, 313.3683664335062, 313.36177724614373, 313.3552059122137, 313.3486523815689, 313.3421166042401, 313.3355985304458, 313.32909811059125, 313.3226152952839, 313.316150035246, 313.3097022812147, 313.30327198411896, 313.2968590950898, 313.2904635654603, 313.2840853467658, 313.27772439073135, 313.2715073555354, 313.26518043712775, 313.25887063793147, 313.25257791008227, 313.2463022059013, 313.24004347789486, 313.2338016787546, 313.22757676132005, 313.2213686784654, 313.21517738322365, 313.2090028288008, 313.2028449685754, 313.1967037561083, 313.1905791450939, 313.18447108922123, 313.1783795423464, 313.1723044585118, 313.16624579194587, 313.1602034970632, 313.1541775284622])
TL_Julia = np.array([324.43747965338866, 324.65119431964285, 324.85975954789393, 325.06335769977835, 325.2621619916594, 325.4563370597661, 325.6460395029098, 325.831418355023, 326.01261555557187, 326.18976635331154, 326.36299972028763, 326.5324386821857, 326.69820069751864, 326.86039792899015, 327.01913757197656, 327.17452211188936, 327.326649569009, 327.47561377397795, 327.6215045465993, 327.76440792755903, 327.9016343722314, 328.038862728153, 328.1733395574249, 328.3051376844441, 328.4343270078231, 328.56097465518127, 328.6851451071509, 328.8069003384313, 328.92629991813214, 329.04340114002645, 329.15825911834656, 329.2709268858126, 329.38145550461104, 329.48989413358385, 329.59629012355913, 329.7006891072855, 329.803135054181, 329.90367035190764, 330.0023358878609, 330.0991710885053, 330.1923304579363, 330.2856525429623, 330.3772537867345, 330.46716843209896, 330.5554295422659, 330.64206905748205, 330.72711783546237, 330.8106056936551, 330.89256145653997, 330.9730129839767, 331.0519872157301, 331.129510207794, 331.2056071584857, 331.2803024456994, 331.3536196629183, 331.4255816361434, 331.49621045769425, 331.5655275199657, 331.63355353175035, 331.70030854260074, 331.76451404413586, 331.82880917662186, 331.8918893969097, 331.95377236985604, 332.01447520798916, 332.07401449497814, 332.13240630342045, 332.1896662093131, 332.24580931254104, 332.30085025232285, 332.3548032188969, 332.40768197161015, 332.45949985546787, 332.51026980935234, 332.5600043804896, 332.6087157418563, 332.6564157014091, 332.7031157106399, 332.74882687808275, 332.7935599847793, 332.8364595972656, 332.879286700631, 332.92116600269577, 332.96210706664766, 333.00211917186516, 333.0412113244551, 333.0793922666754, 333.116670483925, 333.15305421176225, 333.188551447659, 333.2231699548062, 333.2569172695931, 333.28980070975257, 333.3218273831565, 333.3530041905652, 333.3833378332639, 333.41283481997544, 333.4415014756689, 333.4693439424166, 333.49636818706296, 333.52206369456195, 333.54748480565974, 333.5721044934556, 333.59592808357746, 333.6189607522718, 333.64120753178815, 333.66267331609066, 333.6833628648411, 333.7032808070969, 333.7224316469142, 333.7408197687523, 333.7584494387194, 333.7753248101939, 333.79144992809023, 333.8068287342322, 333.8214650682716, 333.8353626728004, 333.84852519599707, 333.86095619896696, 333.872659156748, 333.8834249737009, 333.89369632817585, 333.90324951178854, 333.9120876903375, 333.9202139599134, 333.92763135018924, 333.9343428282658, 333.9403513016712, 333.94565962079076, 333.95027058129693, 333.95418692959197, 333.95741136395486, 333.95994653652724, 333.961795056943, 333.9629594958272, 333.96344238812753, 333.9632462333575, 333.962373499786, 333.96082662513834, 333.9586080227697, 333.9557843862628, 333.9522427935312, 333.9480365327955, 333.9431679373423, 333.9376393261378, 333.9314530061418, 333.92461127506425, 333.91711642411434, 333.9089707393243, 333.9001765035306, 333.8907359995767, 333.88065151325293, 333.86992533337315, 333.85855975474374, 333.8465570793847, 333.8339196205898, 333.820649704809, 333.8067496716479, 333.79222187669313, 333.7770686927065, 333.7616141257566, 333.7452297545065, 333.7282271933089, 333.71060890618406, 333.6923773840827, 333.673535146511, 333.65408474323834, 333.63402875692253, 333.61336980360466, 333.5921105346503, 333.57025363718077, 333.5478018377779, 333.52475790346864, 333.50112464172474, 333.4769049024343, 333.4521015789938, 333.4267176097959, 333.4007559812007, 333.37421972754225, 333.3471119311283, 333.3199947979052, 333.29176463540705, 333.2629724155274, 333.2336214260949, 333.2037150089977, 333.17325656100894, 333.1422495345897, 333.11069743931023, 333.0786038421321, 333.04597236808576, 333.0128067007604, 332.9791105834307, 332.9448878197399, 332.91014227369953, 332.87487787021, 332.8390985955292, 332.8028084976859, 332.7660116871095, 332.7287123366313, 332.6909146815083, 332.65339366614506, 332.6146221081134, 332.57536523873586, 332.5356275432564, 332.49541356976476, 332.4547279290329, 332.4135752943015, 332.3719604010159, 332.32988804669867, 332.2873630905511, 332.2443904530667, 332.2009751152455, 332.15712211851553, 332.1128365643437, 332.06812361322864, 332.0229884841408, 331.977436453344, 331.93147285436015, 331.8851030772832, 331.83833256736614, 331.79211397674067, 331.74456629151643, 331.6966344184581, 331.64832401291494, 331.59964078049944, 331.5505904762169, 331.5011789027332, 331.4514119090062, 331.40129538944154, 331.3508352825592, 331.30003756951226, 331.24890827168156, 331.19745345049506, 331.145679205988, 331.0935916747115, 331.0411970277831, 330.98850146905323, 330.93551123500663, 330.88223259196263, 330.8286718353553, 330.77591467872645, 330.72181400780755, 330.6674501311987, 330.6128294382364, 330.55795833740495, 330.5028432556175, 330.4474906349261, 330.39190693080553, 330.33609861053185, 330.2800721514144, 330.22383403917036, 330.1673907641482, 330.1107488207273, 330.05391470636465, 329.9968949180338, 329.939695950697, 329.88232429424215, 329.82478643333474, 329.76708884483907, 329.7092379961825, 329.6524016920521, 329.5942664155524, 329.53599707034556, 329.47760006343134, 329.4190817812383, 329.3604485895664, 329.3017068306797, 329.2428628206654, 329.183922847836, 329.124893171406, 329.06578001976015, 329.0065895881573, 328.9473280357992, 328.88800148573796, 328.8286160235693, 328.76917769415076, 328.70969250045977, 328.65016640106717, 328.59060531008157, 328.5310150950305, 328.47259403681943, 328.4129632752136, 328.3533205818026, 328.29367162268346, 328.2340220084119, 328.17437729400274, 328.1147429788976, 328.05512450401176, 327.9955272497978, 327.9359565358422, 327.8764176198428, 327.8169156963019, 327.75745589609926, 327.69804328378905, 327.63868285714307, 327.5793795471509, 327.5201382171027, 327.46096366043304, 327.4018606007718, 327.3428336903415, 327.28506561308376, 327.22620292322847, 327.1674298203014, 327.10875066797456, 327.05016975549404, 326.9916912976796, 326.9333194344751, 326.8750582299867, 326.81691167235147, 326.7588836733588, 326.70097806820604, 326.64319861532863, 326.5855489960147, 326.528032814147, 326.4706535962026, 326.4134147912392, 326.3563197707234, 326.2993718285757, 326.2425741812346, 326.1859299676514, 326.1305704505764, 326.0742389932935, 326.0180698653327, 325.9620658987392, 325.9062298489894, 325.85056439505354, 325.79507214002797, 325.73975561145085, 325.68461726168397, 325.6296594680953, 325.574884533663, 325.52029468748304, 325.46589208559607, 325.4116788110386, 325.35765687406774, 325.3038282135467, 325.2501946971599, 325.19675812252825, 325.14352021733276, 325.09048263951945, 325.03870170342617, 324.9860653991236, 324.9336339609837, 324.8814087794847, 324.829391179645, 324.7775824213477, 324.7259837009512, 324.674596151875, 324.6234208452379, 324.57245879069916, 324.52171093740634, 324.47117817537384, 324.42086133569677, 324.3707611911, 324.32087845778926, 324.2712137960752, 324.2217678114419, 324.1725410546339, 324.12353402321133, 324.0747471624301, 324.0271500288, 323.9788002208886, 323.9306716103878, 323.88276444420137, 323.8350789215834, 323.7876151946489, 323.74037337029466, 323.69335351073573, 323.64655563422843, 323.59997971612205, 323.5536256901241, 323.50749344924924, 323.46158284594003, 323.4158936936869, 323.37042576805953, 323.32517880771917, 323.28015251467787, 323.23534655539373, 323.1907605620694, 323.1463941337539, 323.1031276383173, 323.0591946381577, 323.0154798170569, 322.971982649361, 322.92870258015495, 322.8856390257746, 322.8427913756102, 322.80015899244455, 322.75774121310207, 322.7155373494582, 322.67354668953817, 322.6317684980654, 322.59020201667965, 322.5488464655767, 322.5077010441382, 322.4667649316002, 322.4260372872421, 322.3855172517278, 322.3452039479126, 322.3050964816017, 322.2659899901668, 322.2262873782391, 322.1867878420283, 322.14749042519645, 322.10839415733625, 322.0694980543647, 322.03080112002203, 321.99230234598156, 321.954000712314, 321.9158951883281, 321.8779847333343, 321.8402682969261, 321.8027448191628, 321.76541323188945, 321.72827245910156, 321.69132141728875, 321.6545590155846, 321.6179841568493, 321.58159573822303, 321.5453926514387, 321.51009236213093, 321.47425294227526, 321.4385955226558, 321.4031189776777, 321.36782217833036, 321.33270399243736, 321.2977632858094, 321.2629989221732, 321.2284097634206, 321.19399467023504, 321.1597525025765, 321.1256821197475, 321.0917823804758, 321.05805214384816, 321.02449026956924, 320.99109561801293, 320.9578670502945, 320.9248034289729, 320.891903618543, 320.85916648539074, 320.8272408337283, 320.79482246484224, 320.7625634081523, 320.7304625401749, 320.69851874040495, 320.6667308914515, 320.63509787987914, 320.60361859603455, 320.57229193410376, 320.5411167925214, 320.51009207431656, 320.4792166869784, 320.4484895424645, 320.41790955774775, 320.3874756551367, 320.357186762056, 320.3270418110653, 320.2970397401598, 320.2671794933549, 320.2374600203996, 320.20847050897277, 320.1790266918942, 320.14972055346766, 320.1205510674553, 320.0915172137338, 320.0626179783586, 320.03385235415743, 320.00521934053126, 319.976717943362, 319.948347175227, 319.92010605572654, 319.8919936111866, 319.8640088746213, 319.8361508859567, 319.80841869246433, 319.78081134838675, 319.75332791487835, 319.7259674600549, 319.6987290595058, 319.6716117961315, 319.6451535282087, 319.61827343806607, 319.59151179419234, 319.5648677082448, 319.5383402989823, 319.5119286922913, 319.4856320215965, 319.4594494276943, 319.43338005855315, 319.4074230693775, 319.38157762295793, 319.3558428893048, 319.33021804554636, 319.30470227598204, 319.27929477247415, 319.25399473419213, 319.22880136737746, 319.20371388533783, 319.17873150865137, 319.15385346541257, 319.12957347015424, 319.10489975694185, 319.08032811837376, 319.0558578107327, 319.0314880971575, 319.0072182476512, 318.9830475393626, 318.95897525648564, 318.93500068998543, 318.9111231375826, 318.8873419040671, 318.8636563010109, 318.8400656465604, 318.8165692654408, 318.7931664891791, 318.76985665612676, 318.74663911105966, 318.7235132051447, 318.70047829594347, 318.67753374776805, 318.6551351523711, 318.63236766831886, 318.60968868826427, 318.58709760165226, 318.5645938039489, 318.54217669664365, 318.5198456874394, 318.49760019022943, 318.4754396247782, 318.4533634166901, 318.4313709976073, 318.4094618051167, 318.3876352824265, 318.3658908783612, 318.344228047433, 318.32264625005723, 318.30114495216213, 318.2797236250586, 318.2583817454299, 318.2371187954502, 318.2163571884077, 318.19524901298814, 318.1742182560649, 318.1532644210489, 318.13238701634793, 318.11158555536656, 318.0908595566326, 318.07020854384547, 318.0496320455424, 318.0291295950674, 318.00870073065823, 317.9883449955329, 317.9680619375079, 317.9478511089711, 317.9277120668942, 317.9076443730374, 317.88764759377557, 317.8677212998297, 317.84786506625943, 317.8280784724474, 317.8087547740594, 317.7891048437789, 317.7695233259459, 317.7500098171813, 317.73056391810564, 317.7111852333392, 317.6918733715851, 317.6726279457294, 317.6534485725224, 317.6343348725467, 317.6152864702238, 317.59630299401437, 317.5773840760528, 317.5585293520824, 317.53973846145556, 317.5210110472469, 317.50234675631225, 317.4837452389521, 317.4652061488682, 317.44672914315316, 317.4286815846717, 317.41032650816123, 317.3920325147066, 317.37379927499563, 317.3556264628455, 317.3375137552035, 317.3194608322026, 317.30146737729, 317.2835330769448, 317.26565762063905, 317.24784070080455, 317.2300820130644, 317.21238125595175, 317.1947381307964, 317.1771523417249, 317.1596235957007, 317.1421516026957, 317.1247360754207, 317.10737672920953, 317.09007328201955, 317.0731698676925, 317.05597627901034, 317.0388377647492, 317.021754053434, 317.0047248760072, 316.9877499658285, 316.970829058714, 316.9539618930763, 316.9371482096869, 316.9203877516255, 316.9036802642457, 316.887025495363, 316.8704231951065, 316.8538731157521, 316.837375011722, 316.82092863959537, 316.8045337582704, 316.7881901288517, 316.77189751445695, 316.7556556802119, 316.7397877251126, 316.7236457506287, 316.70755386857377, 316.69151185224274, 316.67551947678805, 316.6595765192195, 316.64368275843157, 316.6278379753459, 316.61204195271705, 316.59629447507206, 316.5805953286872, 316.56494430171335, 316.5493411841514, 316.53378576764976, 316.518277845501, 316.5028172126427, 316.48740366576556, 316.4720370033497, 316.4567170254348, 316.4414435335964, 316.42652042263086, 316.411338393881, 316.39620226926064, 316.3811118566608, 316.3660669654015, 316.3510674062317, 316.33611299134844, 316.3212035345349, 316.3063388510062, 316.29151875734203, 316.2767430714732, 316.26201161275503, 316.24732420203037, 316.23268066141776, 316.2180808142996, 316.2035244853223, 316.1890115004513, 316.1745416870971, 316.16011487390733, 316.1457308907115, 316.1316759779738, 316.11737630072446, 316.10311895356085, 316.08890377122106, 316.0747305895497, 316.060599245498, 316.04650957713733, 316.03246142378896, 316.0184546259047, 316.00448902499494, 315.99056446362306, 315.97668078543984, 315.9628378353002, 315.9490354590626, 315.9352735035661, 315.9215518166302, 315.90787024707777, 315.8942286448822, 315.8806268610334, 315.86706474744017, 315.85381222236106, 315.84032822250026, 315.8268834571699, 315.81347778213143, 315.80011105401286, 315.7867831303088, 315.77349386938977, 315.7602431306219, 315.747030774277, 315.7338566614579, 315.72072065409884, 315.7076226149722, 315.6945624078322, 315.6815398972402, 315.668554948528, 315.65560742779803, 315.6426972019307, 315.62982413871043, 315.6169881067832, 315.6041889755232, 315.59168150272154, 315.5789550521533, 315.5662651173748, 315.55361157082405, 315.5409942856272, 315.52841313559924, 315.51586799524995, 315.5033587398925, 315.4908852455783, 315.47844738902126, 315.46604504760137, 315.45367809935544, 315.4413464231254, 315.4290498984194, 315.41678840536093, 315.4045618246888, 315.39237003775844, 315.38021292663126, 315.36809037411246, 315.3560022636013, 315.34418921912066, 315.33216896196205, 315.32018280283785, 315.3082306276209, 315.2963123227403, 315.2844277751813, 315.2725768724899, 315.2607595028697, 315.24897555513724, 315.2372249186466, 315.22550748329394, 315.2138231395017, 315.2021717783541, 315.19055329150194, 315.1789675710957, 315.1674145097862, 315.1558940007244, 315.14440593761356, 315.13295021480207, 315.1215267271419, 315.110362882898, 315.09900291238563, 315.0876748665159, 315.07637864217054, 315.06511413668846, 315.0538812478663, 315.0426798739609, 315.0315099137755, 315.02037126663186, 315.0092638322956, 314.99818751098087, 314.987142203335, 314.97612781054863, 314.96514423430835, 314.95419137671456, 314.9432691402813, 314.93237742793656, 314.9215161430481, 314.91068518953676, 314.8998844717693, 314.8893290112886, 314.87858787960766, 314.86787670101825, 314.85719538158014, 314.846543827736, 314.83592194631154, 314.8253296445173, 314.8147668300241, 314.80423341094945, 314.79372929578363, 314.7832543933947, 314.7728086130151, 314.7623918643254, 314.75200405744897, 314.7416451028589, 314.73131491137684, 314.7210133941728, 314.71074046277636, 314.7004960291827, 314.6902800057973, 314.680295782366, 314.6701357539988, 314.66000387676814, 314.6499000645146, 314.639824231405, 314.6297762919322, 314.6197561609165, 314.6097637535709, 314.59979898549926, 314.5898617726244, 314.57995203119185, 314.57006967776, 314.56021462925935, 314.55038680302107, 314.54058611667807, 314.5308124881619, 314.52106583570253, 314.51134607783166, 314.5016531334672, 314.4919869219122, 314.48253989326827, 314.4729263756719, 314.46333935208855, 314.4537787430584, 314.4442444694036, 314.43473645222815, 314.42525461291905, 314.4157988732027, 314.40636915515165, 314.3969653811161, 314.38758747372606, 314.3782353558843, 314.3689089508054, 314.35960818206905, 314.35033297352123, 314.3410832492674, 314.3318589336724, 314.3226599513611, 314.3134862272757, 314.3043376867204, 314.295396478301, 314.2862975817285, 314.2772236475696, 314.26817460221304, 314.2591503722949, 314.2501508846987, 314.2411760665556, 314.23222584529304, 314.2233001486492, 314.2143989046064, 314.2055220413937, 314.19666948748113, 314.1878411716021, 314.1790370228238, 314.1702569704537, 314.1615009440275, 314.1527688733095, 314.1440606882924, 314.1353763192302, 314.12671569671255, 314.1182512588837, 314.1096374506443, 314.1010471834397, 314.0924803888267, 314.0839369985819, 314.0754169447016, 314.0669201594016, 314.05844657515854, 314.04999612473057, 314.04156874109367, 314.033164357443, 314.02478290719006, 314.0164243239704, 314.0080885417252, 313.99977549461624, 313.99148511700844, 313.9832173434705, 313.97497210877464, 313.9667493479122, 313.95854899617643, 313.95053433067727, 313.9423781589782, 313.9342442048188, 313.9261324043708, 313.9180426940026, 313.9099750102799, 313.901929289965, 313.8939054700518, 313.8859034877911, 313.87792328063, 313.8699647862125, 313.86202794237806, 313.85411268716, 313.84621895886954, 313.8383466960241, 313.8304958373243, 313.82266632165386, 313.8148580880793, 313.8070710758565, 313.79930522450456, 313.791715162394, 313.78399103198734, 313.77628788339547, 313.7686056569468, 313.7609442931481, 313.75330373268406, 313.7456839164177, 313.73808478541866, 313.73050628099236, 313.72294834462303, 313.715410917973, 313.70789394288255, 313.70039736136283, 313.6929211156746, 313.6854651482757, 313.6780294017898, 313.6706138190067, 313.663218342882, 313.655842916539, 313.6484874833246, 313.6412985017337, 313.63398248862273, 313.62668630094703, 313.61940988280753, 313.61215317846734, 313.604916132352, 313.59769868904954, 313.5905007933335, 313.58332239019495, 313.576163424789, 313.56902384243307, 313.56190358860715, 313.5548026089451, 313.5477208493022, 313.5406582557245, 313.5336147744096, 313.52659035170666, 313.5195849341163, 313.51259846829083, 313.5056309010701, 313.498820969607, 313.4918906656237, 313.4849791029193, 313.4780862290393, 313.47121199167844, 313.46435633868003, 313.4575192180365, 313.45070057790804, 313.4439003666557, 313.43711853279234, 313.4303550249797, 313.4236097920284, 313.4168827828905, 313.41017394671167, 313.40348323282404, 313.3968105906994, 313.3901559699484, 313.38351932032106, 313.3769005917063, 313.37029973415105, 313.3638481843146, 313.35728256488824, 313.3507346687042, 313.34420444647054, 313.33769184903247, 313.3311968273724, 313.32471933261024, 313.31825931601816, 313.3118167290545, 313.3053915233187, 313.29898365054737, 313.2925930626145, 313.286219711525, 313.27986354945244, 313.27352452875243, 313.2672026019103, 313.26089772153966, 313.2546098403825, 313.2483389113092, 313.2420848873265, 313.23597230004077, 313.2297516102489, 313.22354768649024, 313.21736048238864, 313.21118995169525, 313.2050360482879, 313.1988987261718, 313.19277793949135, 313.1866736425633, 313.1805857898365, 313.17451433588695, 313.1684592354173, 313.1624204432527, 313.1563979143652, 313.1503916039029, 313.14440146713616, 313.1384274594543, 313.1324695363655, 313.12652765349674, 313.12060176659674, 313.1148098742399, 313.1089155294916, 313.10303704975587, 313.0971743913554, 313.09132751073105, 313.08549636444167, 313.0796809091641, 313.073881101703, 313.0680968990229, 313.0623282582128, 313.0565751364791, 313.05083749114647, 313.045115279654, 313.0394084595691, 313.03371698862827, 313.0280408246844, 313.02237992570105, 313.0167342497522, 313.0111037550225, 313.0054883998076])
yCO2_Julia = np.array([0.1, 0.09955845101405736, 0.09912544881520784, 0.09870063598450751, 0.0982836760226487, 0.09787425184030121, 0.09747206435432926, 0.097076831238508, 0.09668828574038188, 0.09630617563380048, 0.09593026221134301, 0.09556031942052147, 0.09519613298529309, 0.09483749969074905, 0.09448422663528198, 0.09413613059732061, 0.09379303744248982, 0.09345478152356834, 0.09312120521280905, 0.09279215839462986, 0.09247394911667015, 0.0921534550603263, 0.09183708345754428, 0.09152470996944607, 0.09121621584536457, 0.09091148759759349, 0.09061041672080844, 0.0903128994044205, 0.09001883629798932, 0.0897281322537412, 0.08944069611498026, 0.08915644050844748, 0.08887528163184565, 0.08859713909318427, 0.08832193572599681, 0.08804959741790291, 0.08778005297951522, 0.08751323398992716, 0.08724907464907727, 0.08698751167602292, 0.08673364026232874, 0.08647704062864514, 0.08622286249238745, 0.08597105141864926, 0.0857215549517139, 0.08547432251568884, 0.08522930533432407, 0.08498645635064213, 0.08474573014505683, 0.084507082873191, 0.08427047218993851, 0.08403585718441524, 0.0838031983254345, 0.08357245739791938, 0.08334359744333762, 0.08311658271866133, 0.08289137864011883, 0.08266795172997717, 0.0824462695802915, 0.0822263008093491, 0.08201236443198685, 0.08179569936934723, 0.08158065983875193, 0.08136721814201167, 0.08115534742244669, 0.08094502162822446, 0.08073621548237774, 0.08052890445588398, 0.08032306473622833, 0.08011867320162544, 0.07991570739844821, 0.07971414551363412, 0.07951396634982162, 0.07931514930773623, 0.0791176743630631, 0.07892152204228767, 0.07872667340593645, 0.07853311003227816, 0.07834081399683966, 0.07814976785188403, 0.07796373890316878, 0.07777511786889917, 0.07758769742144336, 0.0774014618651397, 0.07721639590924366, 0.07703248465266174, 0.07684971357040714, 0.07666806850218903, 0.07648753564084936, 0.07630810151774577, 0.07612975299472644, 0.07595247725290927, 0.07577626178133712, 0.07560109436575049, 0.0754269630820935, 0.07525385628589454, 0.07508176260280251, 0.07491067091810906, 0.07474057037242067, 0.07457145035177905, 0.07440665403633426, 0.07423944506322326, 0.07407318636365198, 0.07390786822421064, 0.07374348114120642, 0.07358001581350308, 0.07341746313549612, 0.07325581419155612, 0.07309506025055584, 0.07293519275914687, 0.07277620333552351, 0.07261808376625571, 0.07246082599963911, 0.07230442214048091, 0.07214886444424247, 0.07199414531446113, 0.07184025729689181, 0.07168719307565913, 0.07153494546634591, 0.0713835074136598, 0.07123587687579903, 0.07108602141992143, 0.07093695522404414, 0.07078867170660845, 0.0706411643957927, 0.07049442692571294, 0.07034845303238432, 0.0702032365504162, 0.07005877141007223, 0.06991505163429161, 0.0697720713338082, 0.06962982470535178, 0.06948830602920898, 0.06934750966552258, 0.0692074300509973, 0.06906806169571682, 0.06892939918215577, 0.0687914371613144, 0.06865417035134933, 0.06851759353261758, 0.06838441270731098, 0.0682491869082765, 0.06811463589940925, 0.06798075469003109, 0.06784753834140253, 0.06771498196450301, 0.06758308071756225, 0.06745182980366987, 0.06732122446937144, 0.06719126000276851, 0.0670619317308695, 0.06693323501717661, 0.06680516526123582, 0.06667771789617176, 0.06655088838742429, 0.06642467222967942, 0.06629906494526894, 0.06617406208385722, 0.06604965922019734, 0.065925851952954, 0.06580509445793295, 0.06568245356869631, 0.06556039528164789, 0.06543891527283863, 0.06531800923448766, 0.06519767287367188, 0.06507790191097927, 0.06495869207858125, 0.06484003911972376, 0.0647219387872642, 0.06460438684320745, 0.06448737905612525, 0.06437091120036688, 0.06425497905591555, 0.06413957840696241, 0.06402470504105431, 0.063910354748023, 0.06379652331795026, 0.06368320654105232, 0.0635704002075665, 0.06346034117481462, 0.06334853309452125, 0.06323722290174416, 0.06312640637655294, 0.06301607929470072, 0.06290623742702804, 0.06279687653889052, 0.06268799238919431, 0.06257958073013423, 0.06247163730670967, 0.06236415785637138, 0.062257138108270056, 0.0621505737827656, 0.062044460591341354, 0.061938794236249234, 0.06183357041020085, 0.06172878479606008, 0.06162443306642209, 0.06152051088352446, 0.06141701389915578, 0.06131599518143362, 0.06121332722067902, 0.06111107143768788, 0.06100922344163889, 0.06090777883086322, 0.06080673319291296, 0.06070608210468239, 0.06060582113253615, 0.060505945832312905, 0.06040645174955729, 0.06030733441972315, 0.06020858936865965, 0.06011021211255614, 0.06001219815811702, 0.05991454300321915, 0.05981724213718293, 0.05972029104149121, 0.0596236851896872, 0.05952742004773364, 0.05943149107491533, 0.05933780245066233, 0.059242525673155876, 0.059147571498622875, 0.05905293536489193, 0.058958612705786624, 0.05886459895157826, 0.05877088953002266, 0.05867747986712496, 0.058584365387555376, 0.058491541515395505, 0.05839900367497441, 0.05830674729230133, 0.05821476779502503, 0.05812306061322634, 0.058031621180643644, 0.05794044493577411, 0.057849527322904395, 0.05775886379200813, 0.057668449800400756, 0.0575782808130386, 0.0574901485474572, 0.05740045132871099, 0.05731098565960435, 0.05722174704792283, 0.05713273101462659, 0.05704393309413084, 0.05695534883620596, 0.0568669738068918, 0.05677880358934901, 0.05669083378478485, 0.056603060013282566, 0.05651547791599703, 0.05642808315535219, 0.056340871415454574, 0.05625383840410047, 0.056166979853543526, 0.05608029152221908, 0.05599376919465783, 0.055907408682879384, 0.055821205827174834, 0.05573687600661181, 0.05565097315761324, 0.055565215750385095, 0.05547959974996122, 0.05539412115553494, 0.05530877600031802, 0.05522356035306322, 0.05513847031945151, 0.05505350204289801, 0.05496865170516093, 0.054883915527144306, 0.05479928977005837, 0.05471477073700562, 0.05463035477286284, 0.05454603826485219, 0.0544618176441883, 0.05437768938660278, 0.054293650013653674, 0.05420969609258619, 0.054125824237343655, 0.05404370622149594, 0.05395998705371436, 0.05387634014567798, 0.053792762304283606, 0.05370925038727681, 0.05362580130307925, 0.053542412010635705, 0.053459079520723485, 0.05337580089682366, 0.05329257325526386, 0.053209393765577324, 0.05312625965099454, 0.05304316818835132, 0.05296011670944446, 0.05287710260111818, 0.05279412330509011, 0.052711176318194636, 0.05262825919310675, 0.0525453695382466, 0.05246250501843934, 0.052381319978509346, 0.05229849855698811, 0.05221569564532561, 0.05213290913271942, 0.052050136965190105, 0.05196737714539928, 0.051884627732567704, 0.05180188684252997, 0.051719152647732795, 0.0516364233771846, 0.05155369731633213, 0.051470972806711084, 0.05138824824588817, 0.05130552208740613, 0.05122279284059309, 0.05114005907033671, 0.051057319396523915, 0.050974572493832686, 0.050891817091468956, 0.050809051972944407, 0.05072793160968903, 0.05064514387555227, 0.0505623431186058, 0.0504795283344548, 0.050396698570988024, 0.050313852928114175, 0.05023099055688926, 0.05014811065909137, 0.05006521248678338, 0.049982295341971246, 0.04989935857582587, 0.04981640158804528, 0.04973342382619373, 0.049650424785467044, 0.0495674040083148, 0.0494843610830549, 0.049401295643553875, 0.04931820736840193, 0.0492350959806256, 0.049151961247341315, 0.04907046637531514, 0.04898728489693888, 0.04890407962973946, 0.04882085050702837, 0.04873759750231066, 0.048654320628842125, 0.04857101993825788, 0.04848769552009884, 0.04840434750123576, 0.04832097604509239, 0.04823758135073159, 0.04815416365193981, 0.04807072321692017, 0.047987260347725734, 0.04790377537874653, 0.047820268676245184, 0.047736740637592756, 0.04765319169102334, 0.047569622294418946, 0.047486032934438066, 0.047404096488950755, 0.047320469146201166, 0.047236823454357535, 0.047153160007601405, 0.04706947942541987, 0.04698578235208737, 0.0469020694552529, 0.04681834142556743, 0.04673459897610879, 0.04665084284151292, 0.04656707377706473, 0.046483292558100124, 0.046399499979772316, 0.04631569685585708, 0.04623188401797042, 0.04614806231497717, 0.046064232612698146, 0.0459803957931004, 0.045896552753258976, 0.045812704404758715, 0.045730528764818625, 0.04564667264971348, 0.0455628140241742, 0.04547895385206773, 0.04539509310897004, 0.045311232781715595, 0.04522737386718992, 0.04514351737213204, 0.045059664312673615, 0.04497581571357816, 0.044891972607564966, 0.04480813603498805, 0.04472430704359615, 0.044640486687430085, 0.04455667602642491, 0.04447287612604833, 0.04438908805709943, 0.04430531289481532, 0.04422155171830292, 0.04413780561020733, 0.04405575009028974, 0.043972037024636, 0.043888342271507196, 0.04380466692360277, 0.04372101207521588, 0.04363737882192124, 0.043553768259650454, 0.04347018148467123, 0.04338661959329825, 0.043303083681321373, 0.043219574843591874, 0.04313609417390217, 0.04305264276482449, 0.042969221706892886, 0.04288583208841925, 0.04280247499535878, 0.04271915151118542, 0.04263586271622534, 0.042552609687325556, 0.042469393497797464, 0.04238787840455658, 0.04230473830910603, 0.042221638228444325, 0.04213857921923054, 0.042055562333480664, 0.041972588618392793, 0.04188965911568988, 0.04180677486172935, 0.04172393688738412, 0.04164114621766106, 0.04155840387147355, 0.041475710861680816, 0.04139306819502795, 0.041310476871618006, 0.041227937884804694, 0.04114545222125493, 0.041063020860908316, 0.04098064477659119, 0.0408983249337418, 0.040816062290569326, 0.04073550131191122, 0.0406533547226289, 0.04057126814464401, 0.040489242506101364, 0.040407278727482177, 0.04032537772152943, 0.040243540392805845, 0.040161767637868294, 0.040080060345283455, 0.03999841939541238, 0.03991684566026308, 0.03983534000366155, 0.03975390328126131, 0.039672536340269196, 0.03959124001930672, 0.03951001514863029, 0.03942886255014807, 0.03934778303729197, 0.03926677741470262, 0.0391858464785042, 0.03910660738092719, 0.03902582663957717, 0.038945122906625176, 0.03886449694402978, 0.038783949505266646, 0.038703481335313775, 0.03862309317036844, 0.038542785738027004, 0.03846255975739059, 0.038382415938983575, 0.03830235498460921, 0.03822237758760764, 0.03814248443290783, 0.0380626761969423, 0.037982953547450765, 0.03790331714376622, 0.03782376763689653, 0.03774430566953321, 0.03766493187580616, 0.03758564688143746, 0.03750803433528179, 0.03742892697781776, 0.03734991023577418, 0.03727098470211008, 0.037192150961755, 0.03711340959162334, 0.0370347611604356, 0.03695620622886862, 0.03687774534971709, 0.036799379067899506, 0.03672110792030047, 0.03664293243604551, 0.03656485313659544, 0.036486870535751774, 0.036408985139491125, 0.03633119744615552, 0.03625350794663631, 0.03617591712440934, 0.036098425455468555, 0.03602103340822812, 0.03594528629939683, 0.03586809285695378, 0.03579100038935565, 0.03571400933627544, 0.035637120130399505, 0.03556033319745247, 0.03548364895608757, 0.03540706781798817, 0.03533059018805834, 0.03525421646446804, 0.03517794703851852, 0.035101782294848435, 0.035025722611583335, 0.034949768360362864, 0.034873919906264934, 0.03479817760783043, 0.03472254181732261, 0.034647012880776165, 0.03457159113802953, 0.03449627692257246, 0.03442257362990294, 0.034347473278609264, 0.03427248141257887, 0.034197598341327136, 0.03412282436859072, 0.03404815979235351, 0.033973604904782365, 0.03389915999227705, 0.033824825335670904, 0.033750601210277856, 0.03367648788581912, 0.03360248562650843, 0.03352859469125507, 0.03345481533369205, 0.03338114780216899, 0.0333075923396752, 0.033234149184074, 0.033160818568198615, 0.03308760071988276, 0.03301449586193288, 0.03294296293423941, 0.03287008243497698, 0.03279731556082117, 0.03272466251517281, 0.03265212349680171, 0.03257969869986943, 0.032507388313892854, 0.03243519252375187, 0.03236311150988206, 0.03229114544831506, 0.03221929451066189, 0.032147558864095836, 0.032075938671571824, 0.03200443409186148, 0.03193304527956942, 0.03186177238506048, 0.0317906155545699, 0.03171957493036179, 0.0316486506507532, 0.031577842850144505, 0.03150856433913537, 0.0314379875478599, 0.031367527612873, 0.03129718465320951, 0.031226958784262383, 0.031156850117801213, 0.031086858761953384, 0.03101698482118202, 0.0309472283964589, 0.03087758958529992, 0.03080806848178298, 0.030738665176472415, 0.03066937975661729, 0.030600212306200327, 0.03053116290595495, 0.030462231633333274, 0.030393418562498525, 0.030324723764504753, 0.030256147307334596, 0.030187689255920705, 0.0301207153024322, 0.030052491873895137, 0.029984387026514864, 0.029916400813230066, 0.029848533284145592, 0.02978078448654725, 0.02971315446489181, 0.02964564326076863, 0.029578250913046295, 0.029510977457905936, 0.029443822928873518, 0.02937678735672995, 0.029309870769658238, 0.029243073193310522, 0.029176394650821395, 0.029109835162805766, 0.029043394747298487, 0.028977073419891863, 0.02891087119380392, 0.028844788079890944, 0.028780142199106993, 0.028714294950040013, 0.02864856683149313, 0.02858295784501496, 0.028517467989958953, 0.02845209726349511, 0.028386845660605495, 0.028321713174039375, 0.02825669979443105, 0.028191805510335333, 0.028127030308255294, 0.028062374172572666, 0.02799783708562808, 0.027933419027806956, 0.027869119977549966, 0.027804939911360183, 0.027740878803748495, 0.027676936627293498, 0.027613113352738992, 0.027549408949006506, 0.027487093930135365, 0.027423624791808092, 0.02736027442086883, 0.02729704277902778, 0.027233929826297194, 0.027170935521000587, 0.02710805981977114, 0.0270453026775055, 0.026982664047456334, 0.026920143881269003, 0.026857742129001067, 0.026795458739079868, 0.026733293658324987, 0.02667124683204474, 0.026609318204046023, 0.026547507716642257, 0.02648581531062084, 0.026424240925238793, 0.026362784498328625, 0.026301445966316526, 0.026241448523964296, 0.026180343230817197, 0.026119355634766802, 0.026058485666654106, 0.025997733256006485, 0.02593709833104493, 0.025876580818684235, 0.02581618064448928, 0.025755897732745774, 0.025695732006496455, 0.025635683387554363, 0.02557575179648222, 0.025515937152575992, 0.02545623937396081, 0.025396658377602428, 0.025337194079313802, 0.025277846393742677, 0.025218615234330083, 0.025159500513402242, 0.025100502142200866, 0.025042796534355354, 0.024984028269555075, 0.02492537608354095, 0.024866839883230463, 0.024808419574525273, 0.02475011506231705, 0.024691926250488497, 0.024633853041873457, 0.0245758953383101, 0.024518053040676176, 0.024460326048897443, 0.024402714261942366, 0.024345217577783854, 0.02428783589348589, 0.024230569105218095, 0.02417341710826096, 0.02411637979700414, 0.024059457064899845, 0.02400264880451945, 0.023945954907598644, 0.02389050573891024, 0.02383403795901846, 0.023777684215004546, 0.02372144439526408, 0.02366531838740916, 0.023609306078273135, 0.023553407353912176, 0.023497622099569803, 0.023441950199716215, 0.023386391538081964, 0.023330945997663124, 0.023275613460725438, 0.023220393808756633, 0.023165286922539247, 0.023110292682169045, 0.023055410967059365, 0.023000641655943375, 0.022945984626836693, 0.02289143975705758, 0.022837006923282333, 0.022783771324052227, 0.022729559955282294, 0.02267546025122488, 0.022621472085991603, 0.02256759533309388, 0.022513829865446876, 0.02246017555537135, 0.02240663227456263, 0.022353199894118804, 0.02229987828457302, 0.022246667315896654, 0.022193566857507837, 0.022140576778223602, 0.02208769694631617, 0.022034927229535383, 0.021982267495112378, 0.02192971760976285, 0.021877277439663223, 0.0218249468504415, 0.021772725707235378, 0.02172165504098457, 0.02166965020117228, 0.02161775440262734, 0.021565967508541572, 0.02151428938165278, 0.021462719884248055, 0.02141125887816574, 0.021359906224769496, 0.021308661784967584, 0.0212575254192428, 0.021206496987654592, 0.021155576349849277, 0.0211047633650178, 0.021054057891933976, 0.021003459788981403, 0.020952968914156538, 0.020902585125071683, 0.02085230827894255, 0.0208021382325625, 0.02075207484235415, 0.020703116059055487, 0.02065326342312237, 0.020603517013386977, 0.020553876684825008, 0.020504342292076726, 0.020454913689449804, 0.02040559073092128, 0.020356373270115063, 0.020307261160314977, 0.020258254254492743, 0.02020935240530969, 0.020160555465125275, 0.020111863285965596, 0.020063275719543208, 0.02001479261728737, 0.019966413830346693, 0.019918139209591727, 0.019869968605610928, 0.01982190186868007, 0.019773938848798554, 0.019727035570792164, 0.019679277467166828, 0.01963162263208294, 0.01958407091450103, 0.01953662216314302, 0.019489276226494708, 0.019442032952807664, 0.019394892190080632, 0.019347853786067407, 0.01930091758830244, 0.019254083444102368, 0.019207351200572945, 0.019160720704587186, 0.01911419180279045, 0.019067764341632584, 0.019021438167370584, 0.018975213126070867, 0.018929089063608944, 0.018883065825642666, 0.01883714325763069, 0.018792236661867044, 0.01874651296378733, 0.018700889474120137, 0.018655366037621347, 0.018609942498888334, 0.018564618702362094, 0.0185193944923291, 0.01847426971290606, 0.01842924420804363, 0.018384317821550002, 0.018339490397092498, 0.018294761778202807, 0.018250131808262782, 0.018205600330499325, 0.018161167188016223, 0.018116832223797178, 0.01807259528070783, 0.01802845620149702, 0.01798441482877729, 0.01794047100502679, 0.017897500547852187, 0.017853749405953735, 0.017810095343011235, 0.01776653820107886, 0.01772307782211899, 0.01767971404800413, 0.017636446720518607, 0.017593275681345995, 0.017550200772070625, 0.017507221834198287, 0.017464338709157364, 0.017421551238303145, 0.017378859262910894, 0.017336262624162564, 0.01729376116317698, 0.01725135472101367, 0.017209043138674614, 0.01716682625710593, 0.017124703917186027, 0.017082675959715153, 0.01704157997777282, 0.016999738427709864, 0.016957990785421363, 0.016916336891515612, 0.016874776586564992, 0.016833309711107688, 0.016791936105649278, 0.016750655610652733, 0.01670946806653701, 0.016668373313696347, 0.016627371192501655, 0.016586461543303652, 0.016545644206429556, 0.016504919022166404, 0.01646428583078826, 0.01642374447256089, 0.016383294787743342, 0.016342936616589535, 0.016302669799342157, 0.016262494176216013, 0.016223210388096265, 0.01618321485793913, 0.016143310045763686, 0.01610349579178598, 0.016063771936233224, 0.016024138319345307, 0.015984594781376284, 0.015945141162586066, 0.01590577730323746, 0.015866503043613738, 0.015827318224019973, 0.015788222684785257, 0.015749216266262543, 0.015710298808810787, 0.01567147015281763, 0.01563273013870501, 0.015594078606930574, 0.015555515397989104, 0.015517040352410687, 0.015478653310743051, 0.015441119237665944, 0.015402905973476865, 0.015364780238281004, 0.015326741872788516, 0.015288790717759971, 0.015250926614007716, 0.015213149402397222, 0.015175458923840918, 0.015137855019294384, 0.015100337529771322, 0.015062906296344832, 0.015025561160149106, 0.014988301962381757, 0.014951128544285961, 0.014914040747168254, 0.01487703841240537, 0.014840121381445527, 0.014803289495809715, 0.014766542597091653, 0.014729880526942902, 0.014694033846331517, 0.014657539269862671, 0.014621129050526599, 0.014584803030271728, 0.014548561051130142, 0.01451240295521878, 0.0144763285847407, 0.014440337781980192, 0.014404430389297514, 0.014368606249142886, 0.01433286520405812, 0.014297207096677915, 0.014261631769732075, 0.014226139066029795, 0.014190728828472776, 0.014155400900062955, 0.014120155123903676, 0.01408499134320081, 0.014049909401263553, 0.014014909141493571, 0.013980687984127401, 0.013945848993426928, 0.013911091218833075, 0.013876414504156928, 0.013841818693320919, 0.013807303630359926, 0.013772869159422385, 0.013738515124766393, 0.013704241370754494, 0.013670047741865803, 0.01363593408269714, 0.013601900237964112, 0.013567946052504064, 0.013534071371263426, 0.013500276039304766, 0.013466559901815892, 0.013432922804110905, 0.013399364591631217, 0.013365885109946553, 0.013332484204748174, 0.01329982740395886, 0.01326658162542432, 0.013233413964273544, 0.01320032426670127, 0.013167312379036565, 0.013134378147743835, 0.013101521419423827, 0.013068742040810677, 0.013036039858766463, 0.013003414720291788, 0.012970866472526942, 0.012938394962752857, 0.01290600003839345, 0.012873681547006669, 0.01284143933628657, 0.012809273254072953, 0.012777183148352375, 0.01274516886725908, 0.012713230259075952, 0.012681367172231348, 0.012650214471981969, 0.012618500470760099, 0.012586861540044979, 0.012555297528846661, 0.012523808286328495, 0.012492393661808035, 0.01246105350475794, 0.012429787664803751, 0.012398595991718407, 0.012367478335431397, 0.012336434546029954, 0.012305464473759905, 0.012274567969027475, 0.012243744882393246, 0.012212995064570738, 0.012182318366435877, 0.012151714639028059, 0.012121183733551009, 0.012090725501373625, 0.012060339794028993, 0.01203063202148859, 0.012000389475912254, 0.011970219013685942, 0.011940120486989524, 0.011910093748171447, 0.011880138649749595, 0.011850255044412094, 0.011820442785015902, 0.011790701724581536, 0.011761031716300446, 0.011731432613536518, 0.011701904269826863, 0.011672446538883142, 0.01164305927458749, 0.011613742330989106, 0.01158449556231291, 0.011555318822960773, 0.011526211967512275, 0.01149717485072548, 0.011468207327537021, 0.011439886534882076, 0.011411056379717056, 0.011382295386891205, 0.011353603412040855, 0.011324980310983347, 0.011296425939717775, 0.011267940154425749, 0.011239522811470235, 0.011211173767390604, 0.011182892878909046, 0.01115468000293168, 0.01112653499654924, 0.011098457717038155, 0.01107044802185856, 0.011042505768649085, 0.011014630815234489, 0.01098682301962712, 0.010959082240027607, 0.010931408334825568, 0.010903801162600106])
plt.plot(result[5], result[1], label = "Python Code")
plt.plot(TG_Julia, z_Julia*6.394, label = "Julia Code")
plt.legend()
plt.xlabel("Gas Temperature")
plt.ylabel("Column Height (z)")
plt.show()

plt.figure()
plt.plot(result[2]/result[2][0]*0.1, result[1], label = "Python Code")
plt.plot(yCO2_Julia, z_Julia*6.394, label = "Julia Code")
plt.xlabel("CO2 Mole Fraction")
plt.ylabel("Column Height (z)")
plt.legend()


The agreement under these realistic conditions is reasonable. This is unsurprising, as both models use similar constitutive relations; the only difference is the treatment of the axial gas flux: the Julia model treats the N2 flux as constant, while the Python model used in this work treats the total gas flux as constant - a reasonable assumption for absorption from dilute (<10% CO2) streams. 